## Context

## Objective

- Explore and visualize the dataset.
- Build clean the data set and missing values.
- Generate a set of insights from the dataset.

### The key question

Do natural disasters worsen the gender-based nutritional disparity among children in Bangladesh?

### Problem Formulation:

We have a regression problem at hand where we will try to run a diff in diff to see if we can find evidence for a gender bais as.

### Proposed approach

Since it is a regression problem we will first start with the parametric model linear regression with a diff-in-diff approache.

### Overall solution design :

The potential solution design would look like this:

- Checking the data description to get the idea of basic statistics or summary of data.
- Univariate analysis to see how data is spread out, getting to know about the outliers.
- Bivariate analysis to see how different attributes vary with the dependent variable.
- Outlier treatment if needed.
- Missing value treatment using appropriate techniques.
- Feature engineering - transforming features, creating new features if possible.
- Choosing the model evaluation technique - 1) R Squared 2) RMSE can be any other metrics related to regression analysis.
- Splitting the data and proceeding with modeling.


### Measures of success :

R-squared.

## Data Dictionary

## Importing the necessary libraries and overview of the dataset

In [1]:
# Importing the basic libraries we will require for the project

# Import libraries for data manipulation
import pandas as pd
import numpy as np
import os
# Import libraries for data visualization
import matplotlib.pyplot as plt

# Slightly advanced library for data visualization            
import seaborn as sns      

# Import necessary modules
import geopandas as gpd
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

# import module for geoencoding
from geopy.geocoders import Nominatim

# add sleep time
from time import sleep

import logging

# Set up the color sheme:
import mapclassify as mc

# to compute zscores: https://pypi.org/project/cgmzscore/
# Resource R: https://rdrr.io/github/WorldHealthOrganization/anthroplus/man/anthroplus_zscores.html
#from cgmzscore.src.main import z_score_lhfa
#from cgmzscore.src.main import z_score_wfa
#import ast
#https://github.com/ewheeler/pygrowup
#from pygrowup import Observation
#from decimal import Decimal
from dbfread import DBF
import datetime
# Release memory using gc : The gc module to manually trigger garbage collection. 
# Garbage collection is the process of freeing memory that is no longer being used by the program. 
# By manually triggering garbage collection, you can release memory that is no longer needed.
import gc

gc.collect()


20

## Functions

In [2]:
# def return_non_na(curr_data,col):
#     return curr_data[col].replace(['HWHAZWHO', 'HWWAZWHO', 'HWWHZWHO','HWBMIZWHO','Missing'], np.NaN, inplace=True)



def reverse_geocode(geolocator, latlon, sleep_sec):
    """
    This function attempts to perform reverse geocoding using the provided geolocator
    and latitude-longitude coordinates. It handles potential errors and retries on timeouts.

    Args:
        geolocator (object): A geocoding library object used for reverse geocoding.
        latlon (str): A string in the format "latitude,longitude" representing the location.
        sleep_sec (int): The maximum number of seconds to sleep between retries on timeouts.

    Returns:
        object: The result of the reverse geocoding request (may vary depending on the geocoder library).
                - On success, returns the geocoded information.
                - On timeout, retries up to sleep_sec seconds and returns the result.
                - On service error or other exceptions, returns None.
    """

    try:
        # Attempt reverse geocoding using the geolocator in English language
        return geolocator.reverse(latlon, language='en')
    except GeocoderTimedOut:
        # Handle timeout error
        logging.info('TIMED OUT: GeocoderTimedOut: Retrying...')
        # Introduce a random sleep between 1 and sleep_sec seconds to avoid overwhelming the service
        sleep(randint(1 * 100, sleep_sec * 100) / 100)
        # Retry the reverse geocoding with the same parameters
        return reverse_geocode(geolocator, latlon, sleep_sec)
    except GeocoderServiceError as e:
        # Handle service error (e.g., connection refused)
        logging.info('CONNECTION REFUSED: GeocoderServiceError encountered.')
        logging.error(e)  # Log the detailed error for debugging
        return None  # Indicate failure
    except Exception as e:
        # Handle unexpected exceptions
        logging.info('ERROR: Terminating due to exception {}'.format(e))
        return None  # Indicate failure
    

def get_local_information(curr_data, sleep_sec):
    """
    This function enriches a DataFrame with local address information based on latitude and longitude data.

    Args:
        curr_data (pandas.DataFrame): A DataFrame with columns 'LATNUM' and 'LONGNUM' containing latitude and longitude values.
        sleep_sec (int): Number of seconds to sleep between retries for error handling.

    Returns:
        pandas.DataFrame: The DataFrame with additional columns for city, village, county, state_district, state, and country.
        """

    # Initialize the Nominatim geocoder with a user agent string (important for API usage guidelines)
    user_agent = 'bibouPaultest_{}'.format(randint(10000, 99999)) + '@gmail.com'
    geolocator = Nominatim(user_agent=user_agent)

    # Iterate through each row of the DataFrame
    for curr_line in range(curr_data.index.max() + 1):

        # Extract latitude and longitude values for the current row
        Latitude = str(curr_data.loc[curr_line, "LATNUM"])
        Longitude = str(curr_data.loc[curr_line, "LONGNUM"])

        # Perform reverse geocoding to get address information
        location = reverse_geocode(geolocator, Latitude + "," + Longitude, sleep_sec)  # Uses a custom function for retries

        # Extract relevant address components from the geocoding result
        if location:
            address = location.raw['address']
            city = address.get('city')
            village = address.get('village')
            county = address.get('county')
            state_district = address.get('state_district')
            state = address.get('state')
            country = address.get('country')

        # Update the DataFrame with the extracted address information
            curr_data.loc[curr_line, 'city'] = city
            curr_data.loc[curr_line, 'village'] = village
            curr_data.loc[curr_line, 'county'] = county
            curr_data.loc[curr_line, 'state_district'] = state_district
            curr_data.loc[curr_line, 'state'] = state
            curr_data.loc[curr_line, 'country'] = country

        else:
            # Update the DataFrame with the extracted address information
            curr_data.loc[curr_line, 'city'] = np.NAN
            curr_data.loc[curr_line, 'village'] = np.NAN
            curr_data.loc[curr_line, 'county'] = np.NAN
            curr_data.loc[curr_line, 'state_district'] = np.NAN
            curr_data.loc[curr_line, 'state'] = np.NAN
            curr_data.loc[curr_line, 'country'] = np.NAN
    
    # Return the DataFrame with the added address information
    return curr_data


def correct_id(curr_row, nbr_max):
    """
    This function ensures that the input string `curr_row` is formatted to a specified length `nbr_max` 
    by adding leading zeros if necessary.

    Parameters:
    curr_row (str): The current row ID that needs to be corrected.
    nbr_max (int): The maximum length that the corrected ID should have.
    """
    # Check if the length of the current row ID is less than the maximum allowed length
    if len(curr_row) < nbr_max:
        # Calculate the number of leading zeros needed to reach the maximum length
        nbr_resid = nbr_max - len(curr_row) 
        # Add the leading zeros to the beginning of the current row ID and return as a string
        if curr_row.count('.') == 0:
            return str(nbr_resid * '0' + curr_row)
        else:
            # Add extra leading zeros if the current row ID contains a period (.)
            return str((nbr_resid + 2) * '0' + curr_row)
    # Check if the length of the current row ID is exactly equal to the maximum allowed length
    elif len(curr_row) == nbr_max:
        # Return the current row ID as a string, adding two leading zeros if it contains a period
        if curr_row.count('.') == 0:
            return str(curr_row)
        else:
            return str(2 * '0' + curr_row)
    # Case where the length of the current row ID is greater than the maximum allowed length
    else: 
        # Check if the period is in the third position
        if curr_row.find('.') == 2: 
            return str('00' + curr_row)
        # Check if the period is in the fourth position
        elif curr_row.find('.') == 3: 
            return str('0' + curr_row)
        # Return the current row ID as is
        else: 
            return str(curr_row)
        
def formatNumber(num):
    # Check if the number is an integer (i.e., no fractional part)
    if num % 1 == 0:
        # If it is an integer, convert it to an integer type and return it
        return int(num)
    else:
        # If it is not an integer, return the number as it is
        return num
    
def vertical_mean_line_survived(x, **kwargs):
    """
    This function draws a vertical line on a plot at the mean value of the input data 'x'.
    The style and color of the line are determined by additional keyword arguments.

    Parameters:
    x (array-like): The data for which the mean will be calculated.
    **kwargs: Additional keyword arguments to customize the line.
        'label' (str): A label that can be 'male' or 'female' to determine the line style.
        'color' (str): The color of the line, default is green ('g').

    Example usage:
    vertical_mean_line_survived(data, label='male', color='b')
    """

    # Define line styles for male and female
    ls = {"male": "-", "female": "-"}
    
    # Draw a vertical line at the mean of 'x'
    plt.axvline(x.mean(), 
                linestyle=ls[kwargs.get("label")],  # Set line style based on 'label' keyword argument
                color=kwargs.get("color", "g"))     # Set line color based on 'color' keyword argument, default is green

# Example of how the function might be called:
# vertical_mean_line_survived(data, label='male', color='blue')

## Loading the data

### Round 1 data

In [3]:
# Round 1 datasets

## HH identification data: Reading household identification data from a Stata file
df_r1 = pd.read_stata(os.getcwd() + '\\input\\individual_panel_data\\data\\BHIS round 1 (2011-2012)\\001_mod_a_male.dta') 
df_r1_f = pd.read_stata(os.getcwd() + '\\input\\individual_panel_data\\data\\BHIS round 1 (2011-2012)\\002_mod_a_female.dta') 

## FTF: Feed the Futur (FTF) census data from a Stata file
df_r1_ftf = pd.read_stata(os.getcwd() + '\\input\\individual_panel_data\\data\\BHIS round 1 (2011-2012)\\001_census_ftf.dta') 


### Sociaux economics characteristics
df_r1_SE = pd.read_stata(os.getcwd() + '\\input\\individual_panel_data\\data\\BHIS round 1 (2011-2012)\\003_mod_b1_male.dta') 

## Anthro data: Reading anthropometric data from Stata files
### 1 for all household members
df_r1_anthr1 = pd.read_stata(os.getcwd() + '\\input\\individual_panel_data\\data\\BHIS round 1 (2011-2012)\\045_mod_w1_female.dta') 

### 2 for under five children
df_r1_anthr2 = pd.read_stata(os.getcwd() + '\\input\\individual_panel_data\\data\\BHIS round 1 (2011-2012)\\046_mod_w2_female.dta') 

# Importing the Bangladesh raw map: Reading a shapefile containing administrative boundaries of Bangladesh
#bgd_adm = gpd.read_file(os.getcwd() + '\\input\\shapefile_data\\shapefile_zip\\BGD_adm\\BGD_adm3.shp')


# Crosswalk to merge sociaux economic data to geographic informations
crosswalk = pd.read_excel(os.getcwd() + '\\input\\cross_walk_data_final.xlsx', dtype={'upazila_code':'category','code adm':'category','affected_upazila':'category'})


In [4]:

# Cleaning the first dataframe
## Select columns of interest from the dataframe df_r1
df_r1 = df_r1.loc[:, ['div', 'dcode', 'uzcode', 'uncode','vcode_n', 'div_name', 'District', 
                    'District_Name', 'Upazila', 'Upazila_Name', 'Union', 'Union_Name','a01', 
                    'a02', 'a13', 'a15','a16_dd','a16_mm','a16_yy','Sample_type']]

## Rename specific columns for better readability and understanding
df_r1 = df_r1.rename(columns={'div':'division_code', 'District':'district_code', 'District_Name':'district_name','Upazila':'upazila_code','Upazila_Name':'upazila_name',  'Union':'union_code','Union_Name':'union_name','vcode_n':'village_code','a01': "hh_id",
                    "a02": "census_nbr", "a13": "hh_head_religion", "a15": "hh_ethnic_group",'a16_dd':'first_v_day','a16_mm':'first_v_month','a16_yy':'first_v_year', 'Sample_type':'sample_type'})

## Convert specific columns to categorical data type for optimized performance and memory usage
### Identifiant
df_r1['division_code'] = df_r1['division_code'].astype("int64").astype("string")
df_r1['district_code'] = df_r1['district_code'].astype("int64").astype("string")
df_r1['upazila_code'] = df_r1['upazila_code'].astype("int64").astype("string")
df_r1['village_code'] = df_r1['village_code'].astype("int64").astype("string")
df_r1['hh_id'] = df_r1['hh_id'].astype("int64").astype("string")

df_r1['union_code'] = df_r1['union_code'].astype("int64").astype("string")
df_r1['census_nbr'] = df_r1['census_nbr'].astype("category")
df_r1['sample_type'] = df_r1['sample_type'].astype("category")



#df_r1 = df_r1.loc[(df_r1.sample_type!='FTF additional')]
#df_r1.sample_type.value_counts()

df_r1['first_v_day'] = df_r1['first_v_day'].astype("int64")
df_r1['first_v_month'] = df_r1['first_v_month'].astype("int64")
df_r1['first_v_year'] = df_r1['first_v_year'].astype("int64")

# Filtering to only consider household in the national representative sample
#df_r1 = df_r1.loc[(df_r1.sample_type=='FTF Original') | (df_r1.sample_type=='National Representative')]

# Creating the interview date
df_r1['date_int'] = pd.to_datetime(dict(year=df_r1.first_v_year, month=df_r1.first_v_month, day=df_r1.first_v_day))


# Correcting the id by adding '0' to obtain a unique Id.
## Find the maximum length of the household IDs in the dataframe
nbr_max = df_r1.hh_id.str.len().max()
## Apply the correct_id function to each household ID to ensure they all have the same length and update the 'hh_id' column with the corrected IDs
df_r1["hh_id"] = df_r1.hh_id.apply(lambda row: correct_id(row, nbr_max))

## Find the maximum length of the upazila_code in the dataframe
nbr_max = df_r1.upazila_code.str.len().max()
## Apply the correct_id function to each upazila_code to ensure they all have the same length and update the 'upazila_code' column with the corrected IDs
df_r1["upazila_code"] = df_r1.upazila_code.apply(lambda row: correct_id(row, nbr_max))

## Find the maximum length of the village codes in the dataframe
nbr_max = df_r1.village_code.str.len().max()
## Apply the correct_id function to each village code to ensure they all have the same length and update the 'village_code' column with the corrected codes
df_r1["village_code"] = df_r1.village_code.apply(lambda row: correct_id(row, nbr_max))

## Find the maximum length of the union codes in the dataframe
nbr_max = df_r1.union_code.str.len().max()
## Apply the correct_id function to each union code to ensure they all have the same length and update the 'union_code' column with the corrected codes
df_r1["union_code"] = df_r1.union_code.apply(lambda row: correct_id(row, nbr_max))

# Build an unique IDs for each hh
df_r1["hh_id_cmplt"] = df_r1.apply(lambda row : str(row.loc["union_code"]) + str(row.loc["village_code"]) + str(row.loc["hh_id"]), axis=1)

# To build a variable for the current round
df_r1["survey_round"] = '1'

In [5]:
# Cleaning the first dataframe
## Select columns of interest from the dataframe df_r1
df_r1_f = df_r1_f.loc[:, ['a01', 'div', 'dcode', 'uzcode', 'uncode','vcode_n', 'div_name', 'district', 
                    'district_name', 'upazila', 'upazila_name', 'union', 'union_name',
                    'a02', 'a13', 'a15','a16_dd','a16_mm','a16_yy']]

## Rename specific columns for better readability and understanding
df_r1_f = df_r1_f.rename(columns={'a01': "hh_id",'div':'division', "a02": "census_nbr", "a13": "hh_head_religion", "a15": "hh_ethnic_group", 'district':'district_code', 'district_name':'district_name',
                    'upazila':'upazila_code','vcode_n':'village_code', 'upazila_name':'upazila_name', 'union':'union_code', 'union_name':'union_name','a16_dd':'first_v_day','a16_mm':'first_v_month','a16_yy':'first_v_year'})

## Convert specific columns to categorical data type for optimized performance and memory usage
df_r1_f['hh_id'] = df_r1_f['hh_id'].astype("int64").astype("string")
df_r1_f['division'] = df_r1_f['division'].astype("int64").astype("string")
df_r1_f['village_code'] = df_r1_f['village_code'].astype("int64").astype("string")
df_r1_f['census_nbr'] = df_r1_f['census_nbr'].astype("category")
#df_r1_f['sample_type'] = df_r1_f['sample_type'].astype("category")

df_r1_f['first_v_day'] = df_r1_f['first_v_day'].astype("int64")
df_r1_f['first_v_month'] = df_r1_f['first_v_month'].astype("int64")
df_r1_f['first_v_year'] = df_r1_f['first_v_year'].astype("int64")

# Filtering to only consider household in the national representative sample
#df_r1_f = df_r1_f.loc[(df_r1_f.sample_type=='ftf original') | (df_r1_f.sample_type=='national representative')]
# creating the interview date
#df_r1_f['date_int'] = pd.to_datetime(dict(year=df_r1_f.first_v_year, month=df_r1_f.first_v_month, day=df_r1_f.first_v_day))

# Correcting the id by adding '0' to obtain a unique Id.

## Find the maximum length of the household IDs in the dataframe
nbr_max = df_r1_f.hh_id.str.len().max()
## Apply the correct_id function to each household ID to ensure they all have the same length and update the 'hh_id' column with the corrected IDs
df_r1_f["hh_id"] = df_r1_f.hh_id.apply(lambda row: correct_id(row, nbr_max))

## Find the maximum length of the upazila_code in the dataframe
nbr_max = df_r1_f.upazila_code.str.len().max()
## Apply the correct_id function to each upazila_code to ensure they all have the same length and update the 'upazila_code' column with the corrected IDs
df_r1_f["upazila_code"] = df_r1_f.upazila_code.apply(lambda row: correct_id(row, nbr_max))

## Find the maximum length of the village codes in the dataframe
nbr_max = df_r1_f.village_code.str.len().max()
## Apply the correct_id function to each village code to ensure they all have the same length and update the 'village_code' column with the corrected codes
df_r1_f["village_code"] = df_r1_f.village_code.apply(lambda row: correct_id(row, nbr_max))

## Find the maximum length of the union codes in the dataframe
nbr_max = df_r1_f.union_code.str.len().max()
## Apply the correct_id function to each union code to ensure they all have the same length and update the 'union' column with the corrected codes
df_r1_f["union_code"] = df_r1_f.union_code.apply(lambda row: correct_id(row, nbr_max))

# Build an unique IDs for each hh
df_r1_f["hh_id_cmplt"] = df_r1_f.apply(lambda row : str(row.loc["union_code"]) + str(row.loc["village_code"]) + str(row.loc["hh_id"]), axis=1)

# To build a variable for the current round
df_r1_f["survey_round"] = '1'

In [6]:
# Cleaning the first dataframe
## Select columns of interest from the dataframe df_r1
df_r1_ftf = df_r1_ftf.loc[:, ['a01', 'division', 'dcode', 'uzcode', 'uncode', 'vcode', 'district', 'village','distri_a', 'upazila', 'upazil_a', 'union', 'union_na','a1_01', 'a1_02', 'a1_03',"a1_04","a1_05","a1_06"]]

## Rename specific columns for better readability and understanding
df_r1_ftf = df_r1_ftf.rename(columns={'a01': "hh_id", 'vcode':'village_name', 'village':'village_code', 'division':'div_name', 'district':'district_code','distri_a':'district_name',
                    'upazila':'upazila_code', 'upazil_a':'upazila_name','union':'union_code', 'union_na':'union_name', "a1_01": "sexe_hhm", "a1_02": "age_hhm",
                    "a1_03":"relation_head_hh","a1_04":"marital_status_hhm","a1_05":"literacy_hhm","a1_06":"education_high"})

## Convert specific columns to categorical data type for optimized performance and memory usage
df_r1_ftf['hh_id'] = df_r1_ftf['hh_id'].astype("int64", errors='ignore').astype("string")
df_r1_ftf['div_name'] = df_r1_ftf['div_name'].astype("string")
df_r1_ftf['village_code'] = df_r1_ftf['village_code'].astype("int64").astype("string")
df_r1_ftf['sexe_hhm'] = df_r1_ftf['sexe_hhm'].astype("category")
df_r1_ftf['age_hhm'] = df_r1_ftf['age_hhm'].astype("int64")
df_r1_ftf['relation_head_hh'] = df_r1_ftf['relation_head_hh'].astype("category")
df_r1_ftf['marital_status_hhm'] = df_r1_ftf['marital_status_hhm'].astype("category")
df_r1_ftf['literacy_hhm'] = df_r1_ftf['literacy_hhm'].astype("category")
df_r1_ftf['education_high'] = df_r1_ftf['education_high'].astype("category")


# Correcting the id by adding '0' to obtain a unique Id.

## Find the maximum length of the household IDs in the dataframe
#nbr_max = df_r1_ftf.hh_id.str.len().max()
## Apply the correct_id function to each household ID to ensure they all have the same length and update the 'hh_id' column with the corrected IDs
###df_r1_ftf["hh_id"] = df_r1_ftf.hh_id.apply(lambda row: correct_id(row, nbr_max))

## Find the maximum length of the village codes in the dataframe
###nbr_max = df_r1_ftf.village_code.str.len().max()
## Apply the correct_id function to each village code to ensure they all have the same length and update the 'village_code' column with the corrected codes
###df_r1_ftf["village_code"] = df_r1_ftf.village_code.apply(lambda row: correct_id(row, nbr_max))

# To build a variable for the current round
df_r1_ftf["survey_round"] = '1'

In [7]:
# Cleaning the first dataframe
## Select columns of interest from the dataframe df_r1
df_r1_anthr1 = df_r1_anthr1.loc[:, ['a01', 'mid', 'w1_01', 'w1_02', 'w1_03','w1_04', 'w1_05']]

## Rename specific columns for better readability and understanding
df_r1_anthr1 = df_r1_anthr1.rename(columns={'a01': "hh_id",'mid':'hhm_id', "w1_01": "are_you_pregnant", "w1_02": "are_you_lactating", 
                            "w1_03": "weight_kg", 'w1_04':'height_cm', 'w1_05':'if_not_measured_why'})

## Convert specific columns to categorical data type for optimized performance and memory usage
df_r1_anthr1['hh_id'] = df_r1_anthr1['hh_id'].astype("int64").astype("string")
df_r1_anthr1['hhm_id'] = df_r1_anthr1['hhm_id'].astype("int64").astype("string")
df_r1_anthr1['are_you_pregnant'] = df_r1_anthr1['are_you_pregnant'].astype("category")
df_r1_anthr1['are_you_lactating'] = df_r1_anthr1['are_you_lactating'].astype("category")
df_r1_anthr1['weight_kg'] = df_r1_anthr1['weight_kg'].astype("float64")
df_r1_anthr1['height_cm'] = df_r1_anthr1['height_cm'].astype("float64")
# computing the BMI
df_r1_anthr1['bmi'] = df_r1_anthr1['weight_kg']*10000/(df_r1_anthr1['height_cm']**2)
df_r1_anthr1['if_not_measured_why'] = df_r1_anthr1['if_not_measured_why'].astype("category")
#df_r1_anthr1['sample_type'] = df_r1_anthr1['sample_type'].astype("category")

# Filtering to only consider household in the national representative sample
#df_r1_anthr1 = df_r1_anthr1.loc[(df_r1_anthr1.sample_type=='ftf original') | (df_r1_anthr1.sample_type=='national representative')].drop(columns=['sample_type'], axis=1)
df_r1_anthr1.reset_index(inplace=True)

# Correcting the id by adding '0' to obtain a unique Id.

## Find the maximum length of the household IDs in the dataframe
nbr_max = df_r1_anthr1.hh_id.str.len().max()
## Apply the correct_id function to each household ID to ensure they all have the same length and update the 'hh_id' column with the corrected IDs
df_r1_anthr1["hh_id"] = df_r1_anthr1.hh_id.apply(lambda row: correct_id(row, nbr_max))

## Find the maximum length of the village codes in the dataframe
nbr_max = df_r1_anthr1.hhm_id.str.len().max()
## Apply the correct_id function to each village code to ensure they all have the same length and update the 'village_code' column with the corrected codes
df_r1_anthr1["hhm_id"] = df_r1_anthr1.hhm_id.apply(lambda row: correct_id(row, nbr_max))

# Build an unique IDs for each hh member
df_r1_anthr1["member_id"] = df_r1_anthr1.apply(lambda row : str(row.loc["hh_id"]) + str(row.loc["hhm_id"]), axis=1)

# Correcting the no measured output
df_r1_anthr1['if_not_measured_why'] = df_r1_anthr1['if_not_measured_why'].replace({"have measured":"have measured","absent":"absent","sick":"sick","others":"other","refused to  give measurement":"refused"})


C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\1704833532.py:41: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df_r1_anthr1['if_not_measured_why'] = df_r1_anthr1['if_not_measured_why'].replace({"have measured":"have measured","absent":"absent","sick":"sick","others":"other","refused to  give measurement":"refused"})


In [8]:
# Cleaning the first dataframe
## Select columns of interest from the dataframe df_r1
df_r1_anthr2 = df_r1_anthr2.loc[:, ['a01', 'mid', 'w2_01', 'w2_02', 'w2_03', 'w2_04', 'w2_05','w2_07','w2_08', 'w2_10']]

## Rename specific columns for better readability and understanding
df_r1_anthr2 = df_r1_anthr2.rename(columns={'a01': "hh_id",'mid':'hhm_id', "w2_01": "mother_id", "w2_02": "childbirth_day", 'w2_03':'if_not_week_month', 'w2_04':'month_birth', 'w2_05':'year_birth',
                            "w2_07": "weight_kg", 'w2_08':'height_cm', 'w2_10':'if_not_measured_why'})

## Convert specific columns to categorical data type for optimized performance and memory usage
df_r1_anthr2['hh_id'] = df_r1_anthr2['hh_id'].astype("int64").astype("string")
df_r1_anthr2['hhm_id'] = df_r1_anthr2['hhm_id'].astype("int64").astype("string")
df_r1_anthr2['mother_id'] = df_r1_anthr2['mother_id'].astype("string")
df_r1_anthr2['childbirth_day'] = df_r1_anthr2['childbirth_day'].astype("int64", errors='ignore')
df_r1_anthr2['month_birth'] = df_r1_anthr2['month_birth'].astype("int64", errors='ignore')
df_r1_anthr2['year_birth'] = df_r1_anthr2['year_birth'].astype("int64", errors='ignore')
df_r1_anthr2['if_not_week_month'] = df_r1_anthr2['if_not_week_month'].astype("category")
df_r1_anthr2['weight_kg'] = df_r1_anthr2['weight_kg'].astype("float64")
df_r1_anthr2['height_cm'] = df_r1_anthr2['height_cm'].astype("float64")
# computing the BMI
df_r1_anthr2['bmi'] = df_r1_anthr2['weight_kg']*10000/(df_r1_anthr2['height_cm']**2)
df_r1_anthr2['if_not_measured_why'] = df_r1_anthr2['if_not_measured_why'].astype("category")
#df_r1_anthr2['sample_type'] = df_r1_anthr2['sample_type'].astype("category")

# Filtering to only consider household in the national representative sample
#df_r1_anthr2 = df_r1_anthr2.loc[(df_r1_anthr2.sample_type=='ftf original') | (df_r1_anthr2.sample_type=='national representative')].drop(columns=['sample_type'], axis=1)
df_r1_anthr2.reset_index(inplace=True)

# Fill nan in child birth day 
df_r1_anthr2.childbirth_day = df_r1_anthr2.childbirth_day.fillna(1)

# some correction
## there is a child who have as day 31 and month september, I correct it to 30 to make sure that the information is well defined
df_r1_anthr2.loc[(df_r1_anthr2.childbirth_day==31) & (df_r1_anthr2.month_birth==9),"childbirth_day"]=30
## there is only 28 days in february 2007
df_r1_anthr2.loc[(df_r1_anthr2.childbirth_day==29) & (df_r1_anthr2.month_birth==2) & (df_r1_anthr2.year_birth==2007),"childbirth_day"]=28

## A child have 0.5 as month of birth, this code replace it by 1
df_r1_anthr2.loc[(df_r1_anthr2.month_birth==0.5),"month_birth"]=1
df_r1_anthr2['date_birth'] = pd.to_datetime(dict(year=df_r1_anthr2.year_birth, month=df_r1_anthr2.month_birth, day=df_r1_anthr2.childbirth_day))



# Correcting the id by adding '0' to obtain a unique Id.
## Find the maximum length of the household IDs in the dataframe
nbr_max = df_r1_anthr2.hh_id.str.len().max()
## Apply the correct_id function to each household ID to ensure they all have the same length and update the 'hh_id' column with the corrected IDs
df_r1_anthr2["hh_id"] = df_r1_anthr2.hh_id.apply(lambda row: correct_id(row, nbr_max))
## Find the maximum length of the village codes in the dataframe
nbr_max = df_r1_anthr2.hhm_id.str.len().max()
## Apply the correct_id function to each village code to ensure they all have the same length and update the 'village_code' column with the corrected codes
df_r1_anthr2["hhm_id"] = df_r1_anthr2.hhm_id.apply(lambda row: correct_id(row, nbr_max))

# Build an unique IDs for each hh member
df_r1_anthr2["member_id"] = df_r1_anthr2.apply(lambda row : str(row.loc["hh_id"]) + str(row.loc["hhm_id"]), axis=1)

# Correcting the no measured output
df_r1_anthr2['if_not_measured_why'] = df_r1_anthr2['if_not_measured_why'].replace({"have measured":"have measured","absent":"absent","sick":"sick","others":"other","refused to  give measurement":"refused"})


C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\2932269341.py:57: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df_r1_anthr2['if_not_measured_why'] = df_r1_anthr2['if_not_measured_why'].replace({"have measured":"have measured","absent":"absent","sick":"sick","others":"other","refused to  give measurement":"refused"})


In [9]:
# Sociaux economics caracteristiques of the household members

## Select columns of interest from the dataframe df_r1
df_r1_SE = df_r1_SE.loc[:, ['a01', 'mid', 'b1_01', 'b1_02', 'b1_03', 'b1_04', 'b1_07','b1_08','b1_09']]

## Rename specific columns for better readability and understanding
df_r1_SE = df_r1_SE.rename(columns={'a01': "hh_id",'mid':'hhm_id', "b1_01": "hhm_sex", "b1_02": "hhm_age", 'b1_03':'relation_hhh', 'b1_04':'marital_status_hhm', 'b1_07':'literacy_hhm',
                            "b1_08": "education_high", 'b1_09':'curr_att_school'})

## Convert specific columns to categorical data type for optimized performance and memory usage
df_r1_SE['hh_id'] = df_r1_SE['hh_id'].astype("int64").astype("string")
df_r1_SE['hhm_id'] = df_r1_SE['hhm_id'].astype("int64").astype("string")
df_r1_SE['hhm_sex'] = df_r1_SE['hhm_sex'].astype("category")
df_r1_SE['hhm_age'] = df_r1_SE['hhm_age'].astype("int64")

df_r1_SE['relation_hhh'] = df_r1_SE['relation_hhh'].astype("category")
df_r1_SE['marital_status_hhm'] = df_r1_SE['marital_status_hhm'].astype("category")
df_r1_SE['literacy_hhm'] = df_r1_SE['literacy_hhm'].astype("category")

df_r1_SE['education_high'] = df_r1_SE['education_high'].astype("category")
df_r1_SE['curr_att_school'] = df_r1_SE['curr_att_school'].astype("category")
#df_r1_SE['sample_type'] = df_r1_SE['sample_type'].astype("category")

# Filtering to only consider household in the national representative sample
#df_r1_SE = df_r1_SE.loc[(df_r1_SE.sample_type=='ftf original') | (df_r1_SE.sample_type=='national representative')].drop(columns=['sample_type'], axis=1)
df_r1_SE.reset_index(inplace=True)

# Correcting the id by adding '0' to obtain a unique Id.
## Find the maximum length of the household IDs in the dataframe
nbr_max = df_r1_SE.hh_id.str.len().max()
## Apply the correct_id function to each household ID to ensure they all have the same length and update the 'hh_id' column with the corrected IDs
df_r1_SE["hh_id"] = df_r1_SE.hh_id.apply(lambda row: correct_id(row, nbr_max))
## Find the maximum length of the village codes in the dataframe
nbr_max = df_r1_SE.hhm_id.str.len().max()
## Apply the correct_id function to each village code to ensure they all have the same length and update the 'village_code' column with the corrected codes
df_r1_SE["hhm_id"] = df_r1_SE.hhm_id.apply(lambda row: correct_id(row, nbr_max))

# Build an unique IDs for each hh member
df_r1_SE["member_id"] = df_r1_SE.apply(lambda row : str(row.loc["hh_id"]) + str(row.loc["hhm_id"]), axis=1)
df_r1_SE["hhm_status"]="Permanent (r1)"

In [10]:
# filtering the crosswalk
crosswalk = crosswalk[['upazila_code','code adm','affected_upazila_all','affected_komen','affected_roanu','affected_mora']]


### Round 2 data

In [11]:
# Round 1 datasets

## HH identification data: Reading household identification data from a Stata file
df_r2 = pd.read_stata(os.getcwd() + '\\input\\individual_panel_data\\data\\BHIS round 2 (2015)\\001_r2_mod_a_male.dta', convert_categoricals=False) 
df_r2_f = pd.read_stata(os.getcwd() + '\\input\\individual_panel_data\\data\\BHIS round 2 (2015)\\002_r2_mod_a_female.dta', convert_categoricals=False) 


## FTF: Feed the Futur (FTF) census data from a Stata file
#df_r2_ftf = pd.read_stata(os.getcwd() + '\\input\\individual_panel_data\\data\\BHIS round 2 (2015)\\001_census_ftf.dta') 


### Sociaux economics characteristics
df_r2_SE = pd.read_stata(os.getcwd() + '\\input\\individual_panel_data\\data\\BHIS round 2 (2015)\\003_r2_male_mod_b1.dta', convert_categoricals=False) 

## Anthro data: Reading anthropometric data from Stata files
### 1 for all household members
df_r2_anthr1 = pd.read_stata(os.getcwd() + '\\input\\individual_panel_data\\data\\BHIS round 2 (2015)\\059_r2_mod_w1_female.dta') 

### 2 for under five children
df_r2_anthr2 = pd.read_stata(os.getcwd() + '\\input\\individual_panel_data\\data\\BHIS round 2 (2015)\\060_r2_mod_w2_female.dta') 


In [12]:
# Cleaning the first dataframe
## Select columns of interest from the dataframe df_r2
df_r2 = df_r2.loc[:, ['a01', 'div', 'dcode', 'uzcode', 'uncode','vcode', 'div_name', 'District', 
                    'District_Name', 'Upazila', 'Upazila_Name', 'Union', 'Union_Name','mouzacode','mouza_name','Village','village_name',
                    'a02', 'a13', 'a15','hh_type','flag_a']]

## Rename specific columns for better readability and understanding
df_r2 = df_r2.rename(columns={'a01': "hh_id",'div':'division', "a02": "census_nbr", "a13": "hh_head_religion", "a15": "hh_ethnic_group", 'District':'district_code', 'District_Name':'district_name',
                    'Upazila':'upazila_code', 'Upazila_Name':'upazila_name', 'Union':'union_code', 'Union_Name':'union_name','Village':'village_code','village_name':'village_name', 
                    'mouzacode':'mauza code','mouza_name':'mauza name','hh_type':'sample_type', 'flag_a':'interview_status'})

# Apply a function to each element in the 'hh_id' column of the DataFrame 'df_r2' to correct the 'hh_id
df_r2['hh_id'] = df_r2['hh_id'].apply(lambda n: n if n % 1 else int(n), convert_dtype=False)

## Convert specific columns to categorical data type for optimized performance and memory usage
df_r2['hh_id'] = df_r2['hh_id'].astype("string")
df_r2['division'] = df_r2['division'].astype("int64").astype("string")
df_r2['village_code'] = df_r2['village_code'].astype("int64").astype("string")
df_r2['union_code'] = df_r2['union_code'].astype("int64").astype("string")
df_r2['interview_status'] = df_r2['interview_status'].astype("int64").astype("string")
df_r2['census_nbr'] = df_r2['census_nbr'].astype("category")
df_r2['sample_type'] = df_r2['sample_type'].astype("category")

# Replace the numeric value by there label in the 'interview_status'.
df_r2['interview_status'] = df_r2['interview_status'].replace({'1':'Complete','2':'Partial','3':'Refused','4':'Not at home','5':'Migrated'})
df_r2['hh_head_religion'] = df_r2['hh_head_religion'].replace({1:'Muslim',2:'Hindu',3:'Christian',4:'Buddhist',5:'Other(specify)'})
df_r2['hh_ethnic_group'] = df_r2['hh_ethnic_group'].replace({1:'Bangali',2:'Bihari',3:'Sawtal',4:'Khasia',5:'Rakhain',6:'Bown',
                                                            7:'Chak',8:'Chakma',9:'Khumi',10:'Kheyanf',11:'Lusai/Pankho',12:'Marma',13:'Mru(Murong)',
                                                            14:'Tonchonga',15:'Tripura',16:'Bonojogi',17:'Others'})

#df_r1 = df_r1.loc[(df_r1.sample_type!='FTF additional')]
#df_r1.sample_type.value_counts()

#df_r2['first_v_day'] = df_r2['first_v_day'].astype("float64")
#df_r2['first_v_month'] = df_r2['first_v_month'].astype("float64")
#df_r2['first_v_year'] = df_r2['first_v_year'].astype("float64")

# Filtering to only consider household in the national representative sample
#df_r1 = df_r1.loc[(df_r1.sample_type=='FTF Original') | (df_r1.sample_type=='National Representative')]

# creating the interview date
#df_r2['date_int'] = pd.to_datetime(dict(year=df_r2.first_v_year, month=df_r2.first_v_month, day=df_r2.first_v_day))

# Correcting the id by adding '0' to obtain a unique Id.
## Find the maximum length of the household IDs in the dataframe
nbr_max = df_r2.hh_id.str.len().max() - 2
## Apply the correct_id function to each household ID to ensure they all have the same length and update the 'hh_id' column with the corrected IDs
df_r2["hh_id"] = df_r2.hh_id.apply(lambda row: correct_id(row, nbr_max))

## Find the maximum length of the village codes in the dataframe
nbr_max = df_r2.village_code.str.len().max()

## Apply the correct_id function to each village code to ensure they all have the same length and update the 'village_code' column with the corrected codes
df_r2["village_code"] = df_r2.village_code.apply(lambda row: correct_id(row, nbr_max))

## Find the maximum length of the upazila_code in the dataframe
nbr_max = df_r2.upazila_code.str.len().max()

## Apply the correct_id function to each upazila_code to ensure they all have the same length and update the 'upazila_code' column with the corrected IDs
df_r2["upazila_code"] = df_r2.upazila_code.apply(lambda row: correct_id(row, nbr_max))

## Find the maximum length of the union codes in the dataframe
nbr_max = df_r2.union_code.str.len().max()
## Apply the correct_id function to each union code to ensure they all have the same length and update the 'union' column with the corrected codes
df_r2["union_code"] = df_r2.union_code.apply(lambda row: correct_id(row, nbr_max))

# Household initial id to get the parent household
df_r2["hh_id_parent"] = df_r2.hh_id.apply(lambda row: row[:row.find('.')] if row.find('.')!=-1 else row)

## hh_split capture hh who didn't split at least once during the study period (1 if don't split and 0 otherwise)
df_r2["hh_split"] = df_r2.apply(lambda row: '1' if row["hh_id_parent"] != row["hh_id"] else '0', axis=1).astype('category')

## hh_split_bis capture infant hh (1 if the hh is an infant hh and 0 otherwise)
df_r2["hh_split_bis"] = df_r2.apply(lambda row: '1' if (row["hh_id_parent"]+".1" != row["hh_id"]) & (row["hh_id_parent"] != row["hh_id"]) else '0', axis=1).astype('category')


# Build an unique IDs for each hh
df_r2["hh_id_cmplt"] = df_r2.apply(lambda row : str(row.loc["village_code"]) + str(row.loc["hh_id"]), axis=1)

# To build a variable for the current round
df_r2["survey_round"] = '2'

# List of all households surveyed in the second round (This is needed to know which household split between R2 and R3)
lst_hh_r2 = np.unique(df_r2.hh_id.values)



C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\3588970700.py:13: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r2['hh_id'] = df_r2['hh_id'].apply(lambda n: n if n % 1 else int(n), convert_dtype=False)


In [13]:
# Cleaning the first dataframe
## Select columns of interest from the dataframe df_r1
df_r2_f = df_r2_f.loc[:, ['a01', 'div', 'dcode', 'uzcode', 'uncode','Village', 'div_name', 'District', 
                    'District_Name', 'Upazila', 'Upazila_Name', 'Union', 'Union_Name',
                    'a02', 'a13', 'a15','hh_type', 'flag_fem_a']]

## Rename specific columns for better readability and understanding
df_r2_f = df_r2_f.rename(columns={'a01': "hh_id",'div':'division', "a02": "census_nbr", "a13": "hh_head_religion", "a15": "hh_ethnic_group", 'District':'district_code', 'District_Name':'district_name',
                    'Upazila':'upazila_code','Village':'village_code', 'Upazila_Name':'upazila_name', 'Union':'union_code', 'Union_Name':'union_name','hh_type':'sample_type','flag_fem_a':'interview_status'})

# Apply a function to each element in the 'hh_id' column of the DataFrame 'df_r2_f' to correct the 'hh_id
df_r2_f['hh_id'] = df_r2_f['hh_id'].apply( lambda n: n if n % 1 else int(n), convert_dtype=False)

## Convert specific columns to categorical data type for optimized performance and memory usage
df_r2_f['hh_id'] = df_r2_f['hh_id'].astype("string")
df_r2_f['division'] = df_r2_f['division'].astype("int64").astype("string")
df_r2_f['village_code'] = df_r2_f['village_code'].astype("int64").astype("string")
df_r2_f['upazila_code'] = df_r2_f['upazila_code'].astype("int64").astype("string")
df_r2_f['union_code'] = df_r2_f['union_code'].astype("int64").astype("string")
df_r2_f['census_nbr'] = df_r2_f['census_nbr'].astype("category")
df_r2_f['sample_type'] = df_r2_f['sample_type'].astype("category")
#df_r2_f['sample_type'] = df_r2_f['sample_type'].astype("category")

# Replace the numeric value by there label in the 'interview_status'.
df_r2_f['interview_status'] = df_r2_f['interview_status'].replace({'1':'Complete','2':'Partial','3':'Refused','4':'Not at home','5':'Migrated'})

# Correcting the id by adding '0' to obtain a unique Id.
## Find the maximum length of the household IDs in the dataframe
nbr_max = df_r2_f.hh_id.str.len().max() - 2
## Apply the correct_id function to each household ID to ensure they all have the same length and update the 'hh_id' column with the corrected IDs
df_r2_f["hh_id"] = df_r2_f.hh_id.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)

## Find the maximum length of the upazila_code in the dataframe
nbr_max = df_r2_f.upazila_code.str.len().max()
## Apply the correct_id function to each upazila_code to ensure they all have the same length and update the 'upazila_code' column with the corrected IDs
df_r2_f["upazila_code"] = df_r2_f.upazila_code.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)

## Find the maximum length of the village codes in the dataframe
nbr_max = df_r2_f.village_code.str.len().max()
## Apply the correct_id function to each village code to ensure they all have the same length and update the 'village_code' column with the corrected codes
df_r2_f["village_code"] = df_r2_f.village_code.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)

## Find the maximum length of the union codes in the dataframe
nbr_max = df_r2_f.union_code.str.len().max()
## Apply the correct_id function to each union code to ensure they all have the same length and update the 'union' column with the corrected codes
df_r2_f["union_code"] = df_r2_f.union_code.apply(lambda row: correct_id(row, nbr_max))

# Household initial id to get the parent household
df_r2_f["hh_id_parent"] = df_r2_f.hh_id.apply(lambda row: row[:row.find('.')] if row.find('.')!=-1 else row)
df_r2_f["hh_split"] = df_r2_f.apply(lambda row: '1' if row["hh_id_parent"] != row["hh_id"] else '0', axis=1).astype('category')
# Build an unique IDs for each hh
df_r2_f["hh_id_cmplt"] = df_r2_f.apply(lambda row : str(row.loc["union_code"]) + str(row.loc["village_code"]) + str(row.loc["hh_id"]), axis=1)

# To build a variable for the current round
df_r2_f["survey_round"] = '2'

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\2577530553.py:12: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r2_f['hh_id'] = df_r2_f['hh_id'].apply( lambda n: n if n % 1 else int(n), convert_dtype=False)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\2577530553.py:31: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r2_f["hh_id"] = df_r2_f.hh_id.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\2577530553.py:36: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r2_f["upazila_code"] = df_r2_f.upazila_code.apply(lambda row

In [14]:
# Cleaning the first dataframe
## Select columns of interest from the dataframe df_r1
df_r2_anthr1 = df_r2_anthr1.loc[:, ['a01', 'mid','w1_01', 'w1_02', 'w1_03','w1_04', 'w1_05']]

## Rename specific columns for better readability and understanding
df_r2_anthr1 = df_r2_anthr1.rename(columns={'a01': "hh_id",'mid':'hhm_id', "w1_01": "are_you_pregnant", "w1_02": "are_you_lactating",
                            "w1_03": "weight_kg", 'w1_04':'height_cm', 'w1_05':'if_not_measured_why'})

# Apply a function to each element in the 'hh_id' column of the DataFrame 'df_r2_anthr1' to correct the 'hh_id
df_r2_anthr1['hh_id'] = df_r2_anthr1['hh_id'].apply(lambda n: n if n % 1 else int(n), convert_dtype=False)


## Convert specific columns to categorical data type for optimized performance and memory usage
df_r2_anthr1['hh_id'] = df_r2_anthr1['hh_id'].astype("string")
df_r2_anthr1['hhm_id'] = df_r2_anthr1['hhm_id'].astype("int64").astype("string")
df_r2_anthr1['are_you_pregnant'] = df_r2_anthr1['are_you_pregnant'].astype("category")
df_r2_anthr1['are_you_lactating'] = df_r2_anthr1['are_you_lactating'].astype("category")
df_r2_anthr1['weight_kg'] = df_r2_anthr1['weight_kg'].astype("float64")
df_r2_anthr1['height_cm'] = df_r2_anthr1['height_cm'].astype("float64")

# computing the BMI
df_r2_anthr1['bmi'] = df_r2_anthr1['weight_kg']*10000/(df_r2_anthr1['height_cm']**2)

df_r2_anthr1['if_not_measured_why'] = df_r2_anthr1['if_not_measured_why'].astype("category")
df_r2_anthr1['if_not_measured_why'] = df_r2_anthr1['if_not_measured_why'].cat.rename_categories({"Have measured":"have measured","Absent":"absent","Sick":"sick","Other":"other","Refused to  give Measure ment":"refused"})
df_r2_anthr1['if_not_measured_why'] = df_r2_anthr1['if_not_measured_why'].cat.set_categories(["have measured","absent","sick","other","refused","missing"])

# Filtering to only consider household in the national representative sample
##df_r1_anthr2 = df_r1_anthr2.loc[(df_r1_anthr2.sample_type=='ftf original') | (df_r1_anthr2.sample_type=='national representative')]
## df_r1_anthr2.reset_index(inplace=True)

# Correcting the id by adding '0' to obtain a unique Id.
## Find the maximum length of the household IDs in the dataframe
nbr_max = df_r2_anthr1.hh_id.str.len().max() - 2
## Apply the correct_id function to each household ID to ensure they all have the same length and update the 'hh_id' column with the corrected IDs
df_r2_anthr1["hh_id"] = df_r2_anthr1.hh_id.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)
## Find the maximum length of the village codes in the dataframe
nbr_max = df_r2_anthr1.hhm_id.str.len().max()
## Apply the correct_id function to each village code to ensure they all have the same length and update the 'village_code' column with the corrected codes
df_r2_anthr1["hhm_id"] = df_r2_anthr1.hhm_id.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)

# Build an unique IDs for each hh member
df_r2_anthr1["member_id"] = df_r2_anthr1.apply(lambda row : str(row.loc["hh_id"]) + str(row.loc["hhm_id"]), axis=1)

# Correcting the no measured output

df_r2_anthr1['if_not_measured_why'] = df_r2_anthr1['if_not_measured_why'].fillna('missing')

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\3228784964.py:10: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r2_anthr1['hh_id'] = df_r2_anthr1['hh_id'].apply(lambda n: n if n % 1 else int(n), convert_dtype=False)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\3228784964.py:36: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r2_anthr1["hh_id"] = df_r2_anthr1.hh_id.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\3228784964.py:40: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r2_anthr1["hhm_id"] = df_r2_anthr1.hhm_id

In [15]:
# Cleaning the first dataframe
## Select columns of interest from the dataframe df_r1
df_r2_anthr2 = df_r2_anthr2.loc[:, ['a01', 'mid', 'w2_01', 'w2_02', 'w2_03', 'w2_04', 'w2_05','w2_07','w2_08', 'w2_10', 'w2_14']]

## Rename specific columns for better readability and understanding
df_r2_anthr2 = df_r2_anthr2.rename(columns={'a01': "hh_id",'mid':'hhm_id', "w2_01": "mother_id", "w2_02": "childbirth_day", 'w2_03':'if_not_week_month', 'w2_04':'month_birth', 'w2_05':'year_birth',
                            "w2_07": "weight_kg", 'w2_08':'height_cm', 'w2_14':'age_month', 'w2_10':'if_not_measured_why'})

# Filtering to get only informations on underfive children only
df_r2_anthr2 = df_r2_anthr2[df_r2_anthr2.loc[:,'hhm_id'] !='Not under five age child in this household']

# Apply a function to each element in the 'hh_id' column of the DataFrame 'df_r2_anthr2' to correct the 'hh_id
df_r2_anthr2['hh_id'] = df_r2_anthr2['hh_id'].apply( lambda n: n if n % 1 else int(n), convert_dtype=False)

df_r2_anthr2.reset_index(inplace=True)
# Apply encoding to correct label
label_to_int = {label: label for int, label in enumerate(df_r2_anthr2["hhm_id"].unique())}
df_r2_anthr2["hhm_id"] = df_r2_anthr2["hhm_id"].map(label_to_int)

## Convert specific columns to categorical data type for optimized performance and memory usage
df_r2_anthr2['hh_id'] = df_r2_anthr2['hh_id'].astype("string")
df_r2_anthr2['hhm_id'] = df_r2_anthr2['hhm_id'].astype("int64").astype("string")
#df_r2_anthr2['mother_id'] = df_r2_anthr2['mother_id'].astype("int64").astype("string")
#df_r2_anthr2['childbirth_day'] = df_r2_anthr2['childbirth_day'].astype("float64")
#df_r2_anthr2['month_birth'] = df_r2_anthr2['month_birth'].astype("float64")
#df_r2_anthr2['year_birth'] = df_r2_anthr2['year_birth'].astype("float64")
df_r2_anthr2['age_month'] = df_r2_anthr2['age_month'].astype("int64", errors='ignore')
df_r2_anthr2['if_not_week_month'] = df_r2_anthr2['if_not_week_month'].astype("category")
df_r2_anthr2['weight_kg'] = df_r2_anthr2['weight_kg'].astype("float64")
df_r2_anthr2['height_cm'] = df_r2_anthr2['height_cm'].astype("float64")

# computing the BMI
df_r2_anthr2['bmi'] = df_r2_anthr2['weight_kg']*10000/(df_r2_anthr2['height_cm']**2)

df_r2_anthr2['if_not_measured_why'] = df_r2_anthr2['if_not_measured_why'].astype("category")
# Correcting the no measured output
df_r2_anthr2['if_not_measured_why'] = df_r2_anthr2['if_not_measured_why'].cat.rename_categories({"Have measured":"have measured","Absent":"absent","Sick":"sick","Other":"other","Refused to  give Measure ment":"refused"})
df_r2_anthr2['if_not_measured_why'] = df_r2_anthr2['if_not_measured_why'].cat.set_categories(["have measured","absent","sick","other","refused","missing"])

df_r2_anthr2.loc[df_r2_anthr2['age_month']==0,'age_month'] = 1
df_r2_anthr2['age_days'] = df_r2_anthr2['age_month'].apply(lambda curr_age:  curr_age*30)

# Fill Nan in the child birth day
df_r2_anthr2.childbirth_day = df_r2_anthr2.childbirth_day.fillna(1)

# some correction
## there is a child who have as day 31 and month september, I correct it to 30 to make sure that the information is well defined
df_r2_anthr2.loc[(df_r2_anthr2.childbirth_day==31) & (df_r2_anthr2.month_birth==9),"childbirth_day"]=30
## there is only 28 days in february 2007
df_r2_anthr2.loc[(df_r2_anthr2.childbirth_day==29) & (df_r2_anthr2.month_birth==2) & (df_r2_anthr2.year_birth==2007),"childbirth_day"]=28

## A child have 0.5 as month of birth
df_r2_anthr2.loc[(df_r2_anthr2.month_birth==0.5),"month_birth"]=1
df_r2_anthr2['date_birth'] = pd.to_datetime(dict(year=df_r2_anthr2.year_birth, month=df_r2_anthr2.month_birth, day=df_r2_anthr2.childbirth_day))


# Correcting the id by adding '0' to obtain a unique Id.
## Find the maximum length of the household IDs in the dataframe
nbr_max = df_r2_anthr2.hh_id.str.len().max() - 2
## Apply the correct_id function to each household ID to ensure they all have the same length and update the 'hh_id' column with the corrected IDs
df_r2_anthr2["hh_id"] = df_r2_anthr2.hh_id.apply(lambda row: correct_id(row, nbr_max))
## Find the maximum length of the village codes in the dataframe
nbr_max = df_r2_anthr2.hhm_id.str.len().max()
## Apply the correct_id function to each village code to ensure they all have the same length and update the 'village_code' column with the corrected codes
df_r2_anthr2["hhm_id"] = df_r2_anthr2.hhm_id.apply(lambda row: correct_id(row, nbr_max))

# Build an unique IDs for each hh member
df_r2_anthr2["member_id"] = df_r2_anthr2.apply(lambda row : str(row.loc["hh_id"]) + str(row.loc["hhm_id"]), axis=1)


# Replace NaN in the reason for not measured by a give value:
#df_r2_anthr2['if_not_measured_why'] = df_r2_anthr2['if_not_measured_why'].replace({"Have measured":"have measured","Absent":"absent","Sick":"sick","Other":"other","Refused to  give Measure ment":"refused"})

df_r2_anthr2['if_not_measured_why'] = df_r2_anthr2['if_not_measured_why'].fillna("missing")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\3839828607.py:13: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r2_anthr2['hh_id'] = df_r2_anthr2['hh_id'].apply( lambda n: n if n % 1 else int(n), convert_dtype=False)


In [16]:
# Sociaux economics caracteristiques of the household members

## Select columns of interest from the dataframe df_r1
df_r2_SE = df_r2_SE.loc[:, ['a01', 'mid','mem_stat', 'b1_01', 'b1_02', 'b1_03', 'b1_04', 'b1_07','b1_08','b1_09']]

## Rename specific columns for better readability and understanding
df_r2_SE = df_r2_SE.rename(columns={'a01': "hh_id",'mid':'hhm_id', "b1_01": "hhm_sex", "b1_02": "hhm_age", 'b1_03':'relation_hhh', 'b1_04':'marital_status_hhm', 'b1_07':'literacy_hhm',
                            "b1_08": "education_high", 'b1_09':'curr_att_school','mem_stat':'hhm_status'})

# Apply a function to each element in the 'hh_id' column of the DataFrame 'df_r2_SE' to correct the 'hh_id
df_r2_SE['hh_id'] = df_r2_SE['hh_id'].apply( lambda n: n if n % 1 else int(n), convert_dtype=False)

#df_r2_SE['hhm_age'] = df_r2_SE['hhm_age'].fillna(0)
## Convert specific columns to categorical data type for optimized performance and memory usage
df_r2_SE['hh_id'] = df_r2_SE['hh_id'].astype("string")
df_r2_SE['hhm_id'] = df_r2_SE['hhm_id'].astype("int64").astype("string")
df_r2_SE['hhm_sex'] = df_r2_SE['hhm_sex'].astype("category")
df_r2_SE['hhm_age'] = df_r2_SE['hhm_age'].astype("int64", errors='ignore')

df_r2_SE['relation_hhh'] = df_r2_SE['relation_hhh'].astype("category")
df_r2_SE['marital_status_hhm'] = df_r2_SE['marital_status_hhm'].astype("category")
df_r2_SE['literacy_hhm'] = df_r2_SE['literacy_hhm'].astype("category")
#df_r2_SE['hhm_status'] = df_r2_SE['hhm_status'].astype("category")

df_r2_SE['education_high'] = df_r2_SE['education_high'].astype("category")
df_r2_SE['curr_att_school'] = df_r2_SE['curr_att_school'].astype("category")


# Replace the numeric value by there label in the 'interview_status'.
df_r2_SE['relation_hhh'] = df_r2_SE['relation_hhh'].cat.rename_categories({1:'primary respondent',2:'primary respondent husband/wife',3:'son/daughter',4:'daughter/son-in-law',5:'grandson/daughter',6:'father/mother',
                                                            7:'brother/sister',8:'niece/nephew',9:'primary respondent?s cousin',10:'father-in-law/mother-in-law',11:'brother/sister-in-law',
                                                            12:'husband/wife?s  niece/nephew',13:'primary respondent?s husband/wife\'s cousin',14:'other relative',15:'permanent servant',16:'other non relative/friends'})

df_r2_SE['marital_status_hhm'] = df_r2_SE['marital_status_hhm'].cat.rename_categories({1:'unmarried (never married)',2:'married',3:'widow/widower',4:'divorced',5:'separated/deserted'})

df_r2_SE['literacy_hhm'] = df_r2_SE['literacy_hhm'].cat.rename_categories({1:'cannot read and write',2:'can sign only',3:'can read  only',4:'can read and write'})

df_r2_SE['education_high'] = df_r2_SE['education_high'].cat.rename_categories({0:'reads in class i',1:'completed class i',2:'completed class 2',3:'completed class 3',4:'completed class 4',5:'completed class 5',
                                                                6:'completed class 6',7:'completed class 7',8:'completed class 8',9:'completed class 9',10:'completed ssc/dakhil',12:'completed hsc/alim',14:'ba/bsc pass/fazil',
                                                                15:'ba/bsc honors/fazil',16:'ma/msc and above/kamil',22:'ssc candidate',33:'hsc candidate',66:'preschool class (general)',67:'preschool (mosque based)',
                                                                71:'medical/mbbs',72:'nursing',73:'diploma engineer',74:'diploma Enginee',75:'vocational (scolarship based /technical Education',76:'others(specify)',99:'never attended school'})

df_r2_SE['curr_att_school'] = df_r2_SE['curr_att_school'].cat.rename_categories({1:'yes',2:'no'})

df_r2_SE['hhm_status'] = df_r2_SE['hhm_status'].replace({0:'Previous and current round member',1:'New member (new born)',2:'New member through marriage',3:'New member upon return from divorce or seperation',4:'Household merged/combined',5:'Other reasons (permanent)',
                                                                6:'Residing elsewhere for the pursuit of studies',7:'Death',8:'Married and left household',9:'Divorced and left household',10:'Household split',11:'Left household for employment',12:'Other reasns for leaving the household',
                                                                66:'New sample household and current round member'})


# Correcting the id by adding '0' to obtain a unique Id.
## Find the maximum length of the household IDs in the dataframe
nbr_max = df_r2_SE.hh_id.str.len().max() - 2
## Apply the correct_id function to each household ID to ensure they all have the same length and update the 'hh_id' column with the corrected IDs
df_r2_SE["hh_id"] = df_r2_SE.hh_id.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)
## Find the maximum length of the village codes in the dataframe
nbr_max = df_r2_SE.hhm_id.str.len().max()
## Apply the correct_id function to each hh member code to ensure they all have the same length and update the 'hhm_id' column with the corrected codes
df_r2_SE["hhm_id"] = df_r2_SE.hhm_id.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)

# Build an unique IDs for each hh member
df_r2_SE["member_id"] = df_r2_SE.apply(lambda row : str(row.loc["hh_id"]) + str(row.loc["hhm_id"]), axis=1)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\3322896342.py:11: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r2_SE['hh_id'] = df_r2_SE['hh_id'].apply( lambda n: n if n % 1 else int(n), convert_dtype=False)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\3322896342.py:54: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r2_SE["hh_id"] = df_r2_SE.hh_id.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\3322896342.py:58: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r2_SE["hhm_id"] = df_r2_SE.hhm_id.apply(lambda row: corr

In [17]:
df_r2_SE.hhm_status.value_counts()

hhm_status
Previous and current round member                    23210
New sample household and current round member         2387
New member (new born)                                 1676
Household split                                       1410
Other reasons (permanent)                             1358
Left household for employment                          880
Married and left household                             837
New member through marriage                            630
Other reasns for leaving the household                 574
Death                                                  494
Household merged/combined                              442
Residing elsewhere for the pursuit of studies          302
Divorced and left household                             62
New member upon return from divorce or seperation       35
Name: count, dtype: int64

In [18]:
#df_r2_SE.shape
#34297
#df_r2_anthr1.shape
#24591
#df_r2_anthr2.shape
#2763
#24591+2763

#df_r2_SE[df_r2_SE.hhm_status.isna()]
24591+2763 - 34297

-6943

### Round 3 data

#### Import Data

In [19]:
# Round 3 datasets

## HH identification data: Reading household identification data from a Stata file
df_r3 = pd.read_stata(os.getcwd() + '\\input\\individual_panel_data\\data\\BHIS round 3 (2018)\\BIHSRound3\\male\\009_bihs_r3_male_mod_a.dta', convert_categoricals=False) 
df_r3_f = pd.read_stata(os.getcwd() + '\\input\\individual_panel_data\\data\\BHIS round 3 (2018)\\BIHSRound3\\female\\092_bihs_r3_female_mod_a.dta', convert_categoricals=False) 


## FTF: Feed the Futur (FTF) census data from a Stata file
#df_r2_ftf = pd.read_stata(os.getcwd() + '\\input\\individual_panel_data\\data\\BHIS round 2 (2015)\\001_census_ftf.dta') 


### Sociaux economics characteristics
df_r3_SE = pd.read_stata(os.getcwd() + '\\input\\individual_panel_data\\data\\BHIS round 3 (2018)\\BIHSRound3\\male\\010_bihs_r3_male_mod_b1.dta', convert_categoricals=False) 

## Anthro data: Reading anthropometric data from Stata files
### 1 for all household members
df_r3_anthr1 = pd.read_stata(os.getcwd() + '\\input\\individual_panel_data\\data\\BHIS round 3 (2018)\\BIHSRound3\\female\\099_bihs_r3_female_mod_w1.dta') 

### 2 for under five children
df_r3_anthr2 = pd.read_stata(os.getcwd() + '\\input\\individual_panel_data\\data\\BHIS round 3 (2018)\\BIHSRound3\\female\\100_bihs_r3_female_mod_w2.dta') 


#### Male enumerators data cleaning

In [20]:
# Cleaning the first dataframe
## Select columns of interest from the dataframe df_r3
df_r3 = df_r3.loc[:, ['a01', 'div', 'village', 'union', 'div_name', 'district', 'upazila', 'a13', 'a15','a16_1_dd','a16_1_mm','a16_1_yy','a27', 'hh_type']]

## Rename specific columns for better readability and understanding
df_r3 = df_r3.rename(columns={'a01': "hh_id",'div':'division', "a13": "hh_head_religion", "a15": "hh_ethnic_group", 'district':'district_code', 
                    'upazila':'upazila_code','village':'village_code', 'union':'union_code', 'a16_1_dd': 'first_v_day' ,'a16_1_mm': 'first_v_month','a16_1_yy': 'first_v_year','a27':'interview_status','hh_type':'sample_type'})

# Apply a function to each element in the 'hh_id' column of the DataFrame 'df_r3' to correct the 'hh_id
df_r3['hh_id'] = df_r3['hh_id'].apply( lambda n: n if n % 1 else int(n), convert_dtype=False)

## Convert specific columns to categorical data type for optimized performance and memory usage
df_r3['hh_id'] = df_r3['hh_id'].astype("string")
df_r3['division'] = df_r3['division'].astype("int64").astype("string")
df_r3['village_code'] = df_r3['village_code'].astype("int64").astype("string")
df_r3['upazila_code'] = df_r3['upazila_code'].astype("int64").astype("string")
df_r3['union_code'] = df_r3['union_code'].astype("int64").astype("string")
df_r3['interview_status'] = df_r3['interview_status'].astype("category")
df_r3['sample_type'] = df_r3['sample_type'].astype("category")

df_r3['hh_head_religion'] = df_r3['hh_head_religion'].astype("category")
df_r3['hh_ethnic_group'] = df_r3['hh_ethnic_group'].astype("category")
# Replace the numeric value by there label in the 'interview_status'.
df_r3['interview_status'] = df_r3['interview_status'].replace({1:'Complete',2:'Refused',3:'Not at home',4:'Migrated',5:'Partial',6:'Refused'})
df_r3['hh_head_religion'] = df_r3['hh_head_religion'].cat.rename_categories({1:'Muslim',2:'Hindu',3:'Christian',4:'Buddhist',5:'Other(specify)'})
df_r3['hh_ethnic_group'] = df_r3['hh_ethnic_group'].cat.rename_categories({1:'Bangali',2:'Bihari',3:'Sawtal',4:'Khasia',5:'Rakhain',6:'Bown',
                                                            7:'Chak',8:'Chakma',9:'Khumi',10:'Kheyanf',11:'Lusai/Pankho',12:'Marma',13:'Mru(Murong)',
                                                            14:'Tonchonga',15:'Tripura',16:'Bonojogi',17:'Others'})

#df_r1 = df_r1.loc[(df_r1.sample_type!='FTF additional')]
#df_r1.sample_type.value_counts()

#df_r2['first_v_day'] = df_r2['first_v_day'].astype("float64")
#df_r2['first_v_month'] = df_r2['first_v_month'].astype("float64")
#df_r2['first_v_year'] = df_r2['first_v_year'].astype("float64")
df_r3['first_v_day'] = df_r3['first_v_day'].astype("int64")
df_r3['first_v_month'] = df_r3['first_v_month'].astype("int64")
df_r3['first_v_year'] = df_r3['first_v_year'].astype("int64")
# Filtering to only consider household in the national representative sample
#df_r1 = df_r1.loc[(df_r1.sample_type=='FTF Original') | (df_r1.sample_type=='National Representative')]

df_r3['first_v_day'] = df_r3.first_v_day.replace({99999:'01'})
# creating the interview date
df_r3['date_int'] = pd.to_datetime(dict(year=df_r3.first_v_year, month=df_r3.first_v_month, day=df_r3.first_v_day))

# Correcting the id by adding '0' to obtain a unique Id.
## Find the maximum length of the household IDs in the dataframe
nbr_max = df_r3.hh_id.str.len().max() - 3
## Apply the correct_id function to each household ID to ensure they all have the same length and update the 'hh_id' column with the corrected IDs
df_r3["hh_id"] = df_r3.hh_id.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)

## Find the maximum length of the upazila_code in the dataframe
nbr_max = df_r3.upazila_code.str.len().max()
## Apply the correct_id function to each upazila_code to ensure they all have the same length and update the 'upazila_code' column with the corrected IDs
df_r3["upazila_code"] = df_r3.upazila_code.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)

## Find the maximum length of the village codes in the dataframe
nbr_max = df_r3.village_code.str.len().max()
## Apply the correct_id function to each village code to ensure they all have the same length and update the 'village_code' column with the corrected codes
df_r3["village_code"] = df_r3.village_code.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)

## Find the maximum length of the union codes in the dataframe
nbr_max = df_r3.union_code.str.len().max()
## Apply the correct_id function to each union code to ensure they all have the same length and update the 'union' column with the corrected codes
df_r3["union_code"] = df_r3.union_code.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)

# Household initial id to get the parent household
df_r3["hh_id_parent"] = df_r3.hh_id.apply(lambda row: row[:row.find('.')] if row.find('.')!=-1 else row)
df_r3["hh_split"] = df_r3.apply(lambda row: '1' if row["hh_id_parent"] != row["hh_id"] else '0', axis=1).astype('category')

## hh_split_bis capture infant hh (1 if the hh is an infant hh and 0 otherwise)
df_r3["hh_split_bis"] = df_r3.apply(lambda row: '1' if (row["hh_id_parent"]+".1" != row["hh_id"]) & (row["hh_id_parent"] != row["hh_id"]) else '0', axis=1).astype('category')

# Build an unique IDs for each hh
df_r3["hh_id_cmplt"] = df_r3.apply(lambda row : str(row.loc["union_code"]) + str(row.loc["village_code"]) + str(row.loc["hh_id"]), axis=1)

## hh_split between r2 and r3
df_r3["hh_split_btwn_r2_r3"] = df_r3.apply(lambda row: '0' if (row["hh_id"] in lst_hh_r2) else '1', axis=1).astype('category')

# To build a variable for the current round
df_r3["survey_round"] = '3'

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\1472767799.py:10: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r3['hh_id'] = df_r3['hh_id'].apply( lambda n: n if n % 1 else int(n), convert_dtype=False)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\1472767799.py:24: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df_r3['interview_status'] = df_r3['interview_status'].replace({1:'Complete',2:'Refused',3:'Not at home',4:'Migrated',5:'Partial',6:'Refused'})
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\1472767799.py:50: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object

#### Female enumerators data cleaning

In [21]:
# Cleaning the first dataframe
## Select columns of interest from the dataframe df_r1
df_r3_f = df_r3_f.loc[:, ['a01', 'div', 'village', 'union', 'div_name', 'district', 'upazila','a02', 'a13', 'a15', 'a27','hh_type']]

## Rename specific columns for better readability and understanding
df_r3_f = df_r3_f.rename(columns={'a01': "hh_id",'div':'division', "a02": "census_nbr", "a13": "hh_head_religion", "a15": "hh_ethnic_group", 'district':'district_code', 
                    'upazila':'upazila_code','village':'village_code', 'union':'union_code', 'a27':'interview_status', 'hh_type':'sample_type'})

# Apply a function to each element in the 'hh_id' column of the DataFrame 'df_r3_f' to correct the 'hh_id
df_r3_f['hh_id'] = df_r3_f['hh_id'].astype("category").apply( lambda n: n if n % 1 else int(n))

## Convert specific columns to categorical data type for optimized performance and memory usage
df_r3_f['hh_id'] = df_r3_f['hh_id'].astype("string")
df_r3_f['division'] = df_r3_f['division'].astype("int64").astype("string")
df_r3_f['village_code'] = df_r3_f['village_code'].astype("int64").astype("string")
df_r3_f['upazila_code'] = df_r3_f['upazila_code'].astype("int64").astype("string")
df_r3_f['union_code'] = df_r3_f['union_code'].astype("int64").astype("string")
df_r3_f['census_nbr'] = df_r3_f['census_nbr'].astype("category")
df_r3_f['sample_type'] = df_r3_f['sample_type'].astype("category")
df_r3_f['interview_status'] = df_r3_f['interview_status'].astype("category")
# Replace the numeric value by there label in the 'interview_status'.
df_r3_f['interview_status'] = df_r3_f['interview_status'].cat.rename_categories({1:'Complete',2:'Partial',3:'Refused',4:'Not at home',5:'Migrated'})

df_r3_f['hh_head_religion'] = df_r3_f['hh_head_religion'].astype("category")
df_r3_f['hh_ethnic_group'] = df_r3_f['hh_ethnic_group'].astype("category")
# Replace the numeric value by there label
df_r3_f['interview_status'] = df_r3_f['interview_status'].replace({1:'Complete',2:'Refused',3:'Not at home',4:'Migrated',5:'Partial',6:'Refused'})
df_r3_f['hh_head_religion'] = df_r3_f['hh_head_religion'].cat.rename_categories({1:'Muslim',2:'Hindu',3:'Christian',4:'Buddhist',5:'Other(specify)'})
df_r3_f['hh_ethnic_group'] = df_r3_f['hh_ethnic_group'].cat.rename_categories({1:'Bengali',2:'Bihari',3:'Sawtal',4:'Khasia',5:'Rakhain',6:'Bown',
                                                            7:'Chak',8:'Chakma',9:'Khumi',10:'Kheyanf',11:'Lusai/Pankho',12:'Marma',13:'Mru(Murong)',
                                                            14:'Tonchonga',15:'Tripura',16:'Bonojogi',17:'Others'})
# Correcting the id by adding '0' to obtain a unique Id.
## Find the maximum length of the household IDs in the dataframe
nbr_max = df_r3_f.hh_id.str.len().max() - 3
## Apply the correct_id function to each household ID to ensure they all have the same length and update the 'hh_id' column with the corrected IDs
df_r3_f["hh_id"] = df_r3_f.hh_id.astype("category").apply(lambda row: correct_id(row, nbr_max))

## Find the maximum length of the upazila_code in the dataframe
nbr_max = df_r3_f.upazila_code.str.len().max()
## Apply the correct_id function to each upazila_code to ensure they all have the same length and update the 'upazila_code' column with the corrected IDs
df_r3_f["upazila_code"] = df_r3_f.upazila_code.astype("category").apply(lambda row: correct_id(row, nbr_max))

## Find the maximum length of the village codes in the dataframe
nbr_max = df_r3_f.village_code.str.len().max()
## Apply the correct_id function to each village code to ensure they all have the same length and update the 'village_code' column with the corrected codes
df_r3_f["village_code"] = df_r3_f.village_code.astype("category").apply(lambda row: correct_id(row, nbr_max))

## Find the maximum length of the union codes in the dataframe
nbr_max = df_r3_f.union_code.str.len().max()
## Apply the correct_id function to each union code to ensure they all have the same length and update the 'union' column with the corrected codes
df_r3_f["union_code"] = df_r3_f.union_code.astype("category").apply(lambda row: correct_id(row, nbr_max))

# Household initial id to get the parent household
df_r3_f["hh_id_parent"] = df_r3_f.hh_id.apply(lambda row: row[:row.find('.')] if row.find('.')!=-1 else row)
df_r3_f["hh_split"] = df_r3_f.apply(lambda row: '1' if row["hh_id_parent"] != row["hh_id"] else '0', axis=1).astype('category')

# Build an unique IDs for each hh
df_r3_f["hh_id_cmplt"] = df_r3_f.apply(lambda row : str(row.loc["union_code"]) + str(row.loc["village_code"]) + str(row.loc["hh_id"]), axis=1)

# To build a variable for the current round
df_r3_f["survey_round"] = '3'

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\1445982298.py:27: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df_r3_f['interview_status'] = df_r3_f['interview_status'].replace({1:'Complete',2:'Refused',3:'Not at home',4:'Migrated',5:'Partial',6:'Refused'})


#### Anthropometric data (more than 5 years old)

In [22]:
# Cleaning the first dataframe
## Select columns of interest from the dataframe df_r1
df_r3_anthr1 = df_r3_anthr1.loc[:, ['a01', 'mid_w1' ,'w1_01', 'w1_02', 'w1_03','w1_04', 'w1_05']]

## Rename specific columns for better readability and understanding
df_r3_anthr1 = df_r3_anthr1.rename(columns={'a01': "hh_id",'mid_w1':'hhm_id', "w1_01": "are_you_pregnant", "w1_02": "are_you_lactating",'mem_stat':'mem_stat',
                            "w1_03": "weight_kg", 'w1_04':'height_cm', 'w1_05':'if_not_measured_why'})

# Apply a function to each element in the 'hh_id' column of the DataFrame 'df_r3_anthr1' to correct the 'hh_id
df_r3_anthr1['hh_id'] = df_r3_anthr1['hh_id'].apply( lambda n: n if n % 1 else int(n), convert_dtype=False)

#df_r3_anthr1['if_not_measured_why'] = df_r3_anthr1['if_not_measured_why'].cat.set_categories({"have measured","absent","sick","other","refused","missing"})
## Convert specific columns to categorical data type for optimized performance and memory usage
df_r3_anthr1['hh_id'] = df_r3_anthr1['hh_id'].astype("string")
df_r3_anthr1['hhm_id'] = df_r3_anthr1['hhm_id'].astype("int64").astype("string")
df_r3_anthr1['are_you_pregnant'] = df_r3_anthr1['are_you_pregnant'].astype("category")
df_r3_anthr1['are_you_lactating'] = df_r3_anthr1['are_you_lactating'].astype("category")
df_r3_anthr1['weight_kg'] = df_r3_anthr1['weight_kg'].astype("float64")
df_r3_anthr1['height_cm'] = df_r3_anthr1['height_cm'].astype("float64")

# computing the BMI
df_r3_anthr1['bmi'] = df_r3_anthr1['weight_kg']*10000/(df_r3_anthr1['height_cm']**2)

df_r3_anthr1['if_not_measured_why'] = df_r3_anthr1['if_not_measured_why'].astype("category")
# Correcting the no measured output
df_r3_anthr1['if_not_measured_why'] = df_r3_anthr1['if_not_measured_why'].cat.rename_categories({"Have measured":"have measured","Absent":"absent","Sick":"sick","Other (specify)":"other","Refused to  give measurement":"refused"})
df_r3_anthr1['if_not_measured_why'] = df_r3_anthr1['if_not_measured_why'].cat.set_categories({"have measured","absent","sick","other","refused","missing"})

# Correcting the id by adding '0' to obtain a unique Id.
## Find the maximum length of the household IDs in the dataframe
nbr_max = df_r3_anthr1.hh_id.str.len().max() - 3
## Apply the correct_id function to each household ID to ensure they all have the same length and update the 'hh_id' column with the corrected IDs
df_r3_anthr1["hh_id"] = df_r3_anthr1.hh_id.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)
## Find the maximum length of the village codes in the dataframe
nbr_max = df_r3_anthr1.hhm_id.str.len().max()
## Apply the correct_id function to each hh member code to ensure they all have the same length and update the 'hhm_id' column with the corrected codes
df_r3_anthr1["hhm_id"] = df_r3_anthr1.hhm_id.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)

# Build an unique IDs for each hh member
df_r3_anthr1["member_id"] = df_r3_anthr1.apply(lambda row : str(row.loc["hh_id"]) + str(row.loc["hhm_id"]), axis=1)

df_r3_anthr1['if_not_measured_why'] = df_r3_anthr1['if_not_measured_why'].fillna("missing")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\2738657749.py:10: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r3_anthr1['hh_id'] = df_r3_anthr1['hh_id'].apply( lambda n: n if n % 1 else int(n), convert_dtype=False)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\2738657749.py:33: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r3_anthr1["hh_id"] = df_r3_anthr1.hh_id.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\2738657749.py:37: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r3_anthr1["hhm_id"] = df_r3_anthr1.hhm_i

#### Anthropometric data (more less than 5 years old)

In [23]:
# Cleaning the first dataframe
## Select columns of interest from the dataframe df_r3
df_r3_anthr2 = df_r3_anthr2.loc[:, ['a01', 'mid_w2', 'w2_01', 'w2_05','w2_07','w2_08', 'w2_10','w2_02','w2_03','w2_04','w2_05','w2_13','w2_15' ,'haz06', 'waz06', 'bmiz06','flag_zs']]

## Rename specific columns for better readability and understanding
df_r3_anthr2 = df_r3_anthr2.rename(columns={'a01': "hh_id",'mid_w2':'hhm_id', "w2_01": "mother_id",'w2_02':'child',"w2_02": "childbirth_day","w2_13":"birth_order", 'w2_03':'if_not_week_month', 'w2_04':'month_birth', 'w2_05':'year_birth',
                                            'haz06':'HAZ', 'waz06':'WAZ', 'bmiz06':'BMIZ',"w2_07": "weight_kg","w2_15":"age_month", 'w2_08':'height_cm', 'w2_10':'if_not_measured_why','flag_zs':'flag'})

# Apply a function to each element in the 'hh_id' column of the DataFrame 'df_r3_anthr2' to correct the 'hh_id
df_r3_anthr2['hh_id'] = df_r3_anthr2['hh_id'].apply( lambda n: n if n % 1 else int(n), convert_dtype=False)

## Filtering to get surveyed household with an underfive child
df_r3_anthr2 = df_r3_anthr2[df_r3_anthr2.hhm_id !='There are no <5 child in this household']
df_r3_anthr2.reset_index(inplace=True)

# Apply encoding to correct label
label_to_int = {label: label for int, label in enumerate(df_r3_anthr2["hhm_id"].unique())}
df_r3_anthr2["hhm_id"] = df_r3_anthr2["hhm_id"].map(label_to_int)

## Convert specific columns to categorical data type for optimized performance and memory usage
df_r3_anthr2['hh_id'] = df_r3_anthr2['hh_id'].astype("string")
df_r3_anthr2['hhm_id'] = df_r3_anthr2['hhm_id'].astype("int64").astype("string")
#df_r3_anthr2['mother_id'] = df_r3_anthr2['mother_id'].astype("int64").astype("string")
#df_r3_anthr2['childbirth_day'] = df_r3_anthr2['childbirth_day'].astype("int64")
#df_r3_anthr2['month_birth'] = df_r3_anthr2['month_birth'].astype("int64")
#df_r3_anthr2['year_birth'] = df_r3_anthr2['year_birth'].astype("int64")
#df_r3_anthr2['if_not_week_month'] = df_r3_anthr2['if_not_week_month'].astype("category")
df_r3_anthr2['weight_kg'] = df_r3_anthr2['weight_kg'].astype("float64")
df_r3_anthr2['height_cm'] = df_r3_anthr2['height_cm'].astype("float64")

# computing the BMI
df_r3_anthr2['bmi'] = df_r3_anthr2['weight_kg']*10000/(df_r3_anthr2['height_cm']**2)
df_r3_anthr2['if_not_measured_why'] = df_r3_anthr2['if_not_measured_why'].astype("category")
# Correcting the no measured output
df_r3_anthr2['if_not_measured_why'] = df_r3_anthr2['if_not_measured_why'].cat.rename_categories({"Have measured":"have measured","Absent":"absent","Sick":"sick","Other (specify)":"other","Refused to  give measurement":"refused"})
df_r3_anthr2['if_not_measured_why'] = df_r3_anthr2['if_not_measured_why'].cat.set_categories({"have measured","absent","sick","other","refused","missing"})

#df_r3_anthr2.loc[df_r3_anthr2['age_month']==0,'age_month'] = 1
df_r3_anthr2['age_days'] = df_r3_anthr2['age_month'].apply(lambda curr_age:  curr_age*30)

# Fill Nan in the child birth day
#df_r3_anthr2.childbirth_day = df_r3_anthr2.childbirth_day.fillna(1)

# some correction
## there is a child who have as day 31 and month september, I correct it to 30 to make sure that the information is well defined
### df_r3_anthr2.loc[(df_r3_anthr2.childbirth_day==31) & (df_r3_anthr2.month_birth==9),"childbirth_day"]=30
## there is only 28 days in february 2007
### df_r3_anthr2.loc[(df_r3_anthr2.childbirth_day==29) & (df_r3_anthr2.month_birth==2) & (df_r3_anthr2.year_birth==2007),"childbirth_day"]=28

## A child have 0.5 as month of birth
###df_r3_anthr2.loc[(df_r3_anthr2.month_birth==0.5),"month_birth"]=1
###df_r3_anthr2['date_birth'] = pd.to_datetime(dict(year=df_r3_anthr2.year_birth, month=df_r3_anthr2.month_birth, day=df_r3_anthr2.childbirth_day))



# Correcting the id by adding '0' to obtain a unique Id.
## Find the maximum length of the household IDs in the dataframe
nbr_max = df_r3_anthr2.hh_id.str.len().max() - 3
## Apply the correct_id function to each household ID to ensure they all have the same length and update the 'hh_id' column with the corrected IDs
df_r3_anthr2["hh_id"] = df_r3_anthr2.hh_id.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)
## Find the maximum length of the village codes in the dataframe
nbr_max = df_r3_anthr2.hhm_id.str.len().max()
## Apply the correct_id function to each hh member code to ensure they all have the same length and update the 'hhm_id' column with the corrected codes
df_r3_anthr2["hhm_id"] = df_r3_anthr2.hhm_id.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)

# Build an unique IDs for each hh member
df_r3_anthr2["member_id"] = df_r3_anthr2.apply(lambda row : str(row.loc["hh_id"]) + str(row.loc["hhm_id"]), axis=1)

df_r3_anthr2['if_not_measured_why'] = df_r3_anthr2['if_not_measured_why'].fillna("missing")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\4161378374.py:10: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r3_anthr2['hh_id'] = df_r3_anthr2['hh_id'].apply( lambda n: n if n % 1 else int(n), convert_dtype=False)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\4161378374.py:60: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r3_anthr2["hh_id"] = df_r3_anthr2.hh_id.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\4161378374.py:64: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r3_anthr2["hhm_id"] = df_r3_anthr2.hhm_i

#### Other household data (more than 5 years old)

In [24]:
# Sociaux economics caracteristiques of the household members

## Select columns of interest from the dataframe df_r3
df_r3_SE = df_r3_SE.loc[:, ['a01', 'mid', 'mem_stat' ,'b1_01', 'b1_02', 'b1_03', 'b1_04', 'b1_07','b1_08','b1_09']]

## Rename specific columns for better readability and understanding
df_r3_SE = df_r3_SE.rename(columns={'a01': "hh_id",'mid':'hhm_id', "b1_01": "hhm_sex", "b1_02": "hhm_age", 'b1_03':'relation_hhh', 'b1_04':'marital_status_hhm', 'b1_07':'literacy_hhm',
                            "b1_08": "education_high", 'b1_09':'curr_att_school','mem_stat':'hhm_status'})

# Apply a function to each element in the 'hh_id' column of the DataFrame 'df_r3_SE' to correct the 'hh_id
df_r3_SE['hh_id'] = df_r3_SE['hh_id'].apply(lambda n: n if n % 1 else int(n), convert_dtype=False)

## Convert specific columns to categorical data type for optimized performance and memory usage
df_r3_SE['hh_id'] = df_r3_SE['hh_id'].astype("string")
df_r3_SE['hhm_id'] = df_r3_SE['hhm_id'].astype("int64").astype("string")
df_r3_SE['hhm_sex'] = df_r3_SE['hhm_sex'].astype("category")
df_r3_SE['hhm_age'] = df_r3_SE['hhm_age'].astype("int64", errors='ignore')

df_r3_SE['relation_hhh'] = df_r3_SE['relation_hhh'].astype("category")
df_r3_SE['marital_status_hhm'] = df_r3_SE['marital_status_hhm'].astype("category")
df_r3_SE['literacy_hhm'] = df_r3_SE['literacy_hhm'].astype("category")

df_r3_SE['education_high'] = df_r3_SE['education_high'].astype("category")
df_r3_SE['curr_att_school'] = df_r3_SE['curr_att_school'].astype("category")


# Replace the numeric value by there label in the 'interview_status'.
df_r3_SE['relation_hhh'] = df_r3_SE['relation_hhh'].cat.rename_categories({1:'primary respondent',2:'primary respondent husband/wife',3:'son/daughter',4:'daughter/son-in-law',5:'grandson/daughter',6:'father/mother',
                                                            7:'brother/sister',8:'niece/nephew',9:'primary respondent?s cousin',10:'father-in-law/mother-in-law',11:'brother/sister-in-law',
                                                            12:'husband/wife?s  niece/nephew',13:'primary respondent?s husband/wife\'s cousin',14:'other relative',15:'permanent servant',16:'other non relative/friends'})

df_r3_SE['marital_status_hhm'] = df_r3_SE['marital_status_hhm'].cat.rename_categories({1:'unmarried (never married)',2:'married',3:'widow/widower',4:'divorced',5:'separated/deserted'})

df_r3_SE['literacy_hhm'] = df_r3_SE['literacy_hhm'].cat.rename_categories({1:'cannot read and write',2:'can sign only',3:'can read  only',4:'can read and write'})

df_r3_SE['education_high'] = df_r3_SE['education_high'].cat.rename_categories({0:'reads in class i',1:'completed class i',2:'completed class 2',3:'completed class 3',4:'completed class 4',5:'completed class 5',
                                                                6:'completed class 6',7:'completed class 7',8:'completed class 8',9:'completed class 9',10:'completed ssc/dakhil',12:'completed hsc/alim',14:'ba/bsc pass/fazil',
                                                                15:'ba/bsc honors/fazil',16:'ma/msc and above/kamil',22:'ssc candidate',33:'hsc candidate',66:'preschool class (general)',67:'preschool (mosque based)',
                                                                71:'medical/mbbs',72:'nursing',73:'diploma engineer',74:'diploma Enginee',75:'vocational (scolarship based /technical Education',76:'others(specify)',99:'never attended school'})

df_r3_SE['curr_att_school'] = df_r3_SE['curr_att_school'].cat.rename_categories({1:'yes',2:'no'})


df_r3_SE['hhm_status'] = df_r3_SE['hhm_status'].replace({0:'Previous and current round member',1:'New member (new born)',2:'New member through marriage',3:'New member upon return from divorce or seperation',4:'Household merged/combined',5:'Other reasons (permanent)',
                                                                6:'Residing elsewhere for the pursuit of studies',7:'Death',8:'Married and left household',9:'Divorced and left household',10:'Household split',11:'Left household for employment',12:'Other reasns for leaving the household',
                                                                66:'New sample household and current round member'})

#{0:'Prvs and crrnt rnd Mmbr',1:'Nw mmbr (Nw Brn)',2:'Nw mmbr thrgh mrrg',3:'Nw mmbr upon rtrn frm dvrc or sprtn',4:'Hshld mrgd/cmbnd',5:'Other reasons (Permanent)',
#                                                                6:'Residing elsewhere for the pursuit of studies',7:'Death',8:'Married and left household',9:'Divorced and left household',10:'Household split',11:'Left household for employment',12:'Other reasns for leaving the household',
#                                                                66:'New sample Household and current round member'}
# Correcting the id by adding '0' to obtain a unique Id.
## Find the maximum length of the household IDs in the dataframe
nbr_max = df_r3_SE.hh_id.str.len().max() - 3
## Apply the correct_id function to each household ID to ensure they all have the same length and update the 'hh_id' column with the corrected IDs
df_r3_SE["hh_id"] = df_r3_SE.hh_id.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)
## Find the maximum length of the village codes in the dataframe
nbr_max = df_r3_SE.hhm_id.str.len().max()
## Apply the correct_id function to each hh member code to ensure they all have the same length and update the 'hhm_id' column with the corrected codes
df_r3_SE["hhm_id"] = df_r3_SE.hhm_id.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)

# Build an unique IDs for each hh member
df_r3_SE["member_id"] = df_r3_SE.apply(lambda row : str(row.loc["hh_id"]) + str(row.loc["hhm_id"]), axis=1)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\2909285177.py:11: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r3_SE['hh_id'] = df_r3_SE['hh_id'].apply(lambda n: n if n % 1 else int(n), convert_dtype=False)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\2909285177.py:55: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r3_SE["hh_id"] = df_r3_SE.hh_id.apply(lambda row: correct_id(row, nbr_max), convert_dtype=False)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21392\2909285177.py:59: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_r3_SE["hhm_id"] = df_r3_SE.hhm_id.apply(lambda row: corre

In [25]:
df_r3_SE['hhm_status'].value_counts()

hhm_status
Previous and current round member                    19284
Household split                                       3310
New member (new born)                                 1738
Left household for employment                         1603
Married and left household                            1552
Other reasons (permanent)                             1372
Other reasns for leaving the household                1091
Death                                                  993
New member through marriage                            609
Residing elsewhere for the pursuit of studies          552
Household merged/combined                              120
Divorced and left household                             94
New member upon return from divorce or seperation       14
Name: count, dtype: int64

### Building some variales characteristics

There is in the last round of the survey around 6011 households for a total of 5503 distinct household. Which represent an attrition rate of 15.37 percent (not data on the reason for not being surveyed)

#### Round 2

In [26]:
df_r2.interview_status.value_counts()

interview_status
Complete       6436
Migrated        237
Not at home      30
Refused          12
Name: count, dtype: Int64


In the round 2, there is a total of 6715 household survey. Among these, 396 household splitted from the round 1 (around 6 percent of the nbr in the first round): 
* Complete       6436 (99 percent); 6040 (92.88 percent) among the hh who doesn't split ; 6224 distinct hh in r1
* Migrated        237 (3.64 percent); 237 (3.64 percent) ;
* Not at home      30 (0.5 percent); 30 (0.5 percent);
* Refused          12 (0.2 percent); 12 (0.2 percent);

In [27]:
# identifying hh where present in the first round but migrate or leave the house, refused in r2

lst_complete = df_r2[df_r2.interview_status=="Complete"].hh_id_parent.drop_duplicates().values
lst_migrate = df_r2[df_r2.interview_status=="Migrated"].hh_id_parent.drop_duplicates().values
lst_nthome = df_r2[df_r2.interview_status=="Not at home"].hh_id_parent.drop_duplicates().values
lst_refused = df_r2[df_r2.interview_status=="Refused"].hh_id_parent.drop_duplicates().values
lst_split = df_r2[df_r2.hh_split=="1"].hh_id_parent.drop_duplicates().values

df_r1["interview_status_r2"] = df_r1.hh_id.apply(lambda row: "Complete" if row in lst_complete else "Migrated" if row in lst_migrate else "Not at home" if row in lst_nthome else "Refused" if row in lst_refused else "Not found")
df_r1["hh_split_r2"] = df_r1.hh_id.apply(lambda row: "1" if row in lst_split else "0")


#### Round 3

In [28]:
df_r1["hh_split_r2"].value_counts()

hh_split_r2
0    6319
1     184
Name: count, dtype: int64

In [29]:

lst_r3 = np.unique(df_r3[df_r3.hh_split_btwn_r2_r3=="1"].hh_id_parent)
#lst_r2 = np.unique(df_r2[df_r2.hh_split=="1"].hh_id_parent)
# There is 285 household who split between R2 and R3 giving birth to 606 households in the sample.

In the round 3, there is a total of 6011 household survey. Among these, 935 households splitted from the round 2 (around 14.37 percent of the nbr in the first round): 
* Complete       5604 (86.17 percent); 4690 (72.12 percent) among the hh who doesn't split 
* Migrated        360 (5.53 percent); 346 (5.32 percent) 
* Not at home      24 (0.36 percent); 24 (0.36 percent)
* Refused          22 (0.34 percent); 15 (0.34 percent)
* Partial           1 (0.01 percent); 1 (0.01 percent)
* **Not found**: 1421 (22 percent)

In [30]:
# identifying hh who were present in the first round but migrate or leave the house, refused or partial refused

lst_complete = df_r3[df_r3.interview_status=="Complete"].hh_id_parent.drop_duplicates().values
lst_migrate = df_r3[df_r3.interview_status=="Migrated"].hh_id_parent.drop_duplicates().values
lst_nthome = df_r3[df_r3.interview_status=="Not at home"].hh_id_parent.drop_duplicates().values
lst_refused = df_r3[df_r3.interview_status=="Refused"].hh_id_parent.drop_duplicates().values
lst_split = df_r3[df_r3.hh_split=="1"].hh_id_parent.drop_duplicates().values

df_r1["interview_status_r3"] = df_r1.hh_id.apply(lambda row: "Complete" if row in lst_complete else "Migrated" if row in lst_migrate else "Not at home" if row in lst_nthome else "Refused" if row in lst_refused else "Not found")
df_r1["hh_split_r3"] = df_r1.hh_id.apply(lambda row: "1" if row in lst_split else "0")

df_r1["hh_split_btwn_r2_r3"] = df_r1.hh_id.apply(lambda row: "1" if row in lst_r3 else "0")

In [31]:
df_r3[df_r3.hh_split=="1"].groupby("hh_id_parent").size().value_counts()

2    358
3     58
4     10
5      1
Name: count, dtype: int64

In [32]:
# Binary for hh of interest
df_r1["hh_interest"] = df_r1.apply(lambda row: "1" if (row.loc["interview_status_r3"]=="Complete") & (row.loc["hh_split_r2"]=="0") & (row.loc["hh_split_r3"]=="0") else "0", axis=1)


In [33]:
# Binary for the household who split at least one time
df_r1["hh_split_r2_r3"] = df_r1.apply(lambda row: "1" if row["hh_split_r2"]=="1" or row["hh_split_r3"]=="1" else "0", axis=1)
df_r1["hh_split_r2_r3"].value_counts()

hh_split_r2_r3
0    6054
1     449
Name: count, dtype: int64

In [34]:
#df_r1[(df_r1["hh_id"].isin(lst_r3)) & (df_r1["hh_split_r2_r3"]=="1")]
#df_r1[(df_r1["hh_id"].isin(lst_r3)) & (df_r1["hh_id"].isin(lst_r2))].hh_id

In [35]:
# Round 1

## Group the DataFrame 'df_r1_SE' by 'hh_id'
df_r1_SE_gr = df_r1_SE.groupby('hh_id')

## Aggregate the grouped data
# aggregated_result_r1 = df_r1_SE_gr.agg(
#     ## Count the number of females in each household
#     nbr_female=('hhm_sex', lambda x: (x == 'female').sum()),

#     ## Count the number of household members under 5 years old
#     nbr_underfive=('hhm_age', lambda x: (x < 5).sum()),

#     ## Count the number of household members between 5 and 10 years old
#     nbr_yngchldrn_5_10 = ('hhm_age', lambda x: ((x >= 5)  & (x < 10)).sum()),

#     ## Count the number of household members between 10 and 19 years old
#     nbr_teenager_10_19 =('hhm_age', lambda x: ((x >= 10)  & (x < 19)).sum()),

#     ## Count the number of household members between 20 and 65 years old
#     nbr_adults_20_65 = ('hhm_age', lambda x: ((x >= 20)  & (x < 65)).sum()),

#     ## Count the number of household members 65 years old and over
#     nbr_elderly_65_over = ('hhm_age', lambda x: (x >= 65).sum()),
# ).reset_index()


# Round 2

## Group the DataFrame 'df_r1_SE' by 'hh_id'
df_r2_SE_gr = df_r2_SE.groupby('hh_id')

# ## Aggregate the grouped data
# aggregated_result_r2 = df_r2_SE_gr.agg(
#     ## Count the number of females in each household
#     nbr_female=('hhm_sex', lambda x: (x == 'female').sum()),

#     ## Count the number of household members under 5 years old
#     nbr_underfive=('hhm_age', lambda x: (x < 5).sum()),

#     ## Count the number of household members between 5 and 10 years old
#     nbr_yngchldrn_5_10 = ('hhm_age', lambda x: ((x >= 5)  & (x < 10)).sum()),

#     ## Count the number of household members between 10 and 19 years old
#     nbr_teenager_10_19 =('hhm_age', lambda x: ((x >= 10)  & (x < 19)).sum()),

#     ## Count the number of household members between 20 and 65 years old
#     nbr_adults_20_65 = ('hhm_age', lambda x: ((x >= 20)  & (x < 65)).sum()),

#     ## Count the number of household members 65 years old and over
#     nbr_elderly_65_over = ('hhm_age', lambda x: (x >= 65).sum()),
# ).reset_index()


# Round 3

## Group the DataFrame 'df_r1_SE' by 'hh_id'
df_r3_SE_gr = df_r3_SE.groupby('hh_id')

## Aggregate the grouped data
# aggregated_result_r3 = df_r3_SE_gr.agg(
#     ## Count the number of females in each household
#     nbr_female=('hhm_sex', lambda x: (x == 'female').sum()),

#     ## Count the number of household members under 5 years old
#     nbr_underfive=('hhm_age', lambda x: (x < 5).sum()),

#     ## Count the number of household members between 5 and 10 years old
#     nbr_yngchldrn_5_10 = ('hhm_age', lambda x: ((x >= 5)  & (x < 10)).sum()),

#     ## Count the number of household members between 10 and 19 years old
#     nbr_teenager_10_19 =('hhm_age', lambda x: ((x >= 10)  & (x < 19)).sum()),

#     ## Count the number of household members between 20 and 65 years old
#     nbr_adults_20_65 = ('hhm_age', lambda x: ((x >= 20)  & (x < 65)).sum()),

#     ## Count the number of household members 65 years old and over
#     nbr_elderly_65_over = ('hhm_age', lambda x: (x >= 65).sum()),
# ).reset_index()

# Display the aggregated result
#aggregated_result_r1

In [36]:
# Round 1
df_r1_SE = (
    df_r1_SE
        # Merge the original dataframe with the aggregated counts of household members by age and sex
        .merge(
                df_r1_SE_gr
                .agg(
                    nbr_female=('hhm_sex', lambda x: (x == 'female').sum()),  # Count of females in the household
                    nbr_underfive=('hhm_age', lambda x: (x < 5).sum()),  # Count of children under 5 years old
                    nbr_yngchldrn_5_10=('hhm_age', lambda x: ((x >= 5) & (x < 10)).sum()),  # Count of children aged 5-10
                    nbr_teenager_10_20=('hhm_age', lambda x: ((x >= 10) & (x < 20)).sum()),  # Count of teenagers aged 10-20
                    nbr_adults_20_65=('hhm_age', lambda x: ((x >= 20) & (x < 65)).sum()),  # Count of adults aged 20-65
                    nbr_elderly_65_over=('hhm_age', lambda x: (x >= 65).sum())  # Count of elderly aged 65 and over
                )
                .reset_index()  # Reset index to prepare for merging
        )
        # Merge with the count of females under 5 years old
        .merge(
            df_r1_SE_gr
            .apply(lambda row: ((row["hhm_sex"]=="female") & (row["hhm_age"]<5)).sum(), include_groups=False)
            .reset_index(name='nbr_female_underfive'), on="hh_id", how="left"
        )
        # Merge with the count of females aged 5-10
        .merge(
            df_r1_SE_gr
            .apply(lambda row: ((row["hhm_sex"]=="female") & (row["hhm_age"]>=5) & (row["hhm_age"] < 10)).sum(), include_groups=False)
            .reset_index(name='nbr_female_yngchldrn_5_10'), on="hh_id", how="left"
        )
        # Merge with the count of females aged 10-20
        .merge(
            df_r1_SE_gr
            .apply(lambda row: ((row["hhm_sex"]=="female") & (row["hhm_age"]>=10) & (row["hhm_age"] < 20)).sum(), include_groups=False)
            .reset_index(name='nbr_female_teenager_10_20'), on="hh_id", how="left"
        )
        # Merge with the count of females aged 20-65
        .merge(
            df_r1_SE_gr
            .apply(lambda row: ((row["hhm_sex"]=="female") & (row["hhm_age"]>=20) & (row["hhm_age"] < 65)).sum(), include_groups=False)
            .reset_index(name='nbr_female_adults_20_65'), on="hh_id", how="left"
        )
        # Merge with the count of females aged 65 and over
        .merge(
            df_r1_SE_gr
            .apply(lambda row: ((row["hhm_sex"]=="female") & (row["hhm_age"]>=65)).sum(), include_groups=False)
            .reset_index(name='nbr_female_elderly_65_over'), on="hh_id", how="left"
        )
)


# Round 2

df_r2_SE = (
    df_r2_SE
        # Merge the original dataframe with the aggregated counts of household members by age and sex
        .merge(
                df_r2_SE_gr
                .agg(
                    nbr_female=('hhm_sex', lambda x: (x == 'female').sum()),  # Count of females in the household
                    nbr_underfive=('hhm_age', lambda x: (x < 5).sum()),  # Count of children under 5 years old
                    nbr_yngchldrn_5_10=('hhm_age', lambda x: ((x >= 5) & (x < 10)).sum()),  # Count of children aged 5-10
                    nbr_teenager_10_20=('hhm_age', lambda x: ((x >= 10) & (x < 20)).sum()),  # Count of teenagers aged 10-20
                    nbr_adults_20_65=('hhm_age', lambda x: ((x >= 20) & (x < 65)).sum()),  # Count of adults aged 20-65
                    nbr_elderly_65_over=('hhm_age', lambda x: (x >= 65).sum())  # Count of elderly aged 65 and over
                )
                .reset_index()  # Reset index to prepare for merging
        )
        # Merge with the count of females under 5 years old
        .merge(
            df_r2_SE_gr
            .apply(lambda row: ((row["hhm_sex"]=="female") & (row["hhm_age"]<5)).sum(), include_groups=False)
            .reset_index(name='nbr_female_underfive'), on="hh_id", how="left"
        )
        # Merge with the count of females aged 5-10
        .merge(
            df_r2_SE_gr
            .apply(lambda row: ((row["hhm_sex"]=="female") & (row["hhm_age"]>=5) & (row["hhm_age"] < 10)).sum(), include_groups=False)
            .reset_index(name='nbr_female_yngchldrn_5_10'), on="hh_id", how="left"
        )
        # Merge with the count of females aged 10-20
        .merge(
            df_r2_SE_gr
            .apply(lambda row: ((row["hhm_sex"]=="female") & (row["hhm_age"]>=10) & (row["hhm_age"] < 20)).sum(), include_groups=False)
            .reset_index(name='nbr_female_teenager_10_20'), on="hh_id", how="left"
        )
        # Merge with the count of females aged 20-65
        .merge(
            df_r2_SE_gr
            .apply(lambda row: ((row["hhm_sex"]=="female") & (row["hhm_age"]>=20) & (row["hhm_age"] < 65)).sum(), include_groups=False)
            .reset_index(name='nbr_female_adults_20_65'), on="hh_id", how="left"
        )
        # Merge with the count of females aged 65 and over
        .merge(
            df_r2_SE_gr
            .apply(lambda row: ((row["hhm_sex"]=="female") & (row["hhm_age"]>=65)).sum(), include_groups=False)
            .reset_index(name='nbr_female_elderly_65_over'), on="hh_id", how="left"
        )
)

# Round 3
df_r3_SE = (
    df_r3_SE
        # Merge the original dataframe with the aggregated counts of household members by age and sex
        .merge(
                df_r3_SE_gr
                .agg(
                    nbr_female=('hhm_sex', lambda x: (x == 'female').sum()),  # Count of females in the household
                    nbr_underfive=('hhm_age', lambda x: (x < 5).sum()),  # Count of children under 5 years old
                    nbr_yngchldrn_5_10=('hhm_age', lambda x: ((x >= 5) & (x < 10)).sum()),  # Count of children aged 5-10
                    nbr_teenager_10_20=('hhm_age', lambda x: ((x >= 10) & (x < 20)).sum()),  # Count of teenagers aged 10-20
                    nbr_adults_20_65=('hhm_age', lambda x: ((x >= 20) & (x < 65)).sum()),  # Count of adults aged 20-65
                    nbr_elderly_65_over=('hhm_age', lambda x: (x >= 65).sum())  # Count of elderly aged 65 and over
                )
                .reset_index()  # Reset index to prepare for merging
        )
        # Merge with the count of females under 5 years old
        .merge(
            df_r3_SE_gr
            .apply(lambda row: ((row["hhm_sex"]=="female") & (row["hhm_age"]<5)).sum(), include_groups=False)
            .reset_index(name='nbr_female_underfive'), on="hh_id", how="left"
        )
        # Merge with the count of females aged 5-10
        .merge(
            df_r3_SE_gr
            .apply(lambda row: ((row["hhm_sex"]=="female") & (row["hhm_age"]>=5) & (row["hhm_age"] < 10)).sum(), include_groups=False)
            .reset_index(name='nbr_female_yngchldrn_5_10'), on="hh_id", how="left"
        )
        # Merge with the count of females aged 10-20
        .merge(
            df_r3_SE_gr
            .apply(lambda row: ((row["hhm_sex"]=="female") & (row["hhm_age"]>=10) & (row["hhm_age"] < 20)).sum(), include_groups=False)
            .reset_index(name='nbr_female_teenager_10_20'), on="hh_id", how="left"
        )
        # Merge with the count of females aged 20-65
        .merge(
            df_r3_SE_gr
            .apply(lambda row: ((row["hhm_sex"]=="female") & (row["hhm_age"]>=20) & (row["hhm_age"] < 65)).sum(), include_groups=False)
            .reset_index(name='nbr_female_adults_20_65'), on="hh_id", how="left"
        )
        # Merge with the count of females aged 65 and over
        .merge(
            df_r3_SE_gr
            .apply(lambda row: ((row["hhm_sex"]=="female") & (row["hhm_age"]>=65)).sum(), include_groups=False)
            .reset_index(name='nbr_female_elderly_65_over'), on="hh_id", how="left"
        )
)

In [37]:
df_r3_SE.columns

Index(['hh_id', 'hhm_id', 'hhm_status', 'hhm_sex', 'hhm_age', 'relation_hhh',
       'marital_status_hhm', 'literacy_hhm', 'education_high',
       'curr_att_school', 'member_id', 'nbr_female', 'nbr_underfive',
       'nbr_yngchldrn_5_10', 'nbr_teenager_10_20', 'nbr_adults_20_65',
       'nbr_elderly_65_over', 'nbr_female_underfive',
       'nbr_female_yngchldrn_5_10', 'nbr_female_teenager_10_20',
       'nbr_female_adults_20_65', 'nbr_female_elderly_65_over'],
      dtype='object')

#### Cross table

##### For all household

In [38]:
pd.crosstab(df_r1["interview_status_r2"], df_r1["interview_status_r3"], margins=True, rownames=["Interview status R2"], colnames=["Interview status R3"])

Interview status R3,Complete,Migrated,Not at home,Not found,Refused,All
Interview status R2,,,,,,
Complete,5025,200,23,965,11,6224
Migrated,68,137,1,31,0,237
Not at home,17,8,0,4,1,30
Refused,7,1,0,1,3,12
All,5117,346,24,1001,15,6503


##### For the household in the nationally representative sample (for comparaison purpose with the sample the table in the repport of the BIHS)

There are some discrepancies between the report and the dataset that I can't explain:

* In the dataset, the FTF additional sample size is 1,080, whereas the report states it as 1,040.
* The sum of National Representative (4,423) and FTF Original (1,000) samples equals 5,423. However, the report indicates a total of 5,503, meaning 80 households are missing from the dataset compared to the report.

In [39]:
pd.crosstab(df_r1[(df_r1.sample_type == "National Representative") | (df_r1.sample_type =="FTF additional")]["interview_status_r2"], df_r1[(df_r1.sample_type == "National Representative") | (df_r1.sample_type =="FTF additional")]["interview_status_r3"], margins=True, rownames=["Interview status R2"], colnames=["Interview status R3"])

Interview status R3,Complete,Migrated,Not at home,Not found,Refused,All
Interview status R2,,,,,,
Complete,5025,200,23,1,11,5260
Migrated,68,137,1,0,0,206
Not at home,17,8,0,0,1,26
Refused,7,1,0,0,3,11
All,5117,346,24,1,15,5503


## Joining dataframe

In [40]:
## merging with the crosswalk
df_r1 = df_r1.merge(crosswalk, on=['upazila_code'], how='left')
df_r2 = df_r2.merge(crosswalk, on=['upazila_code'], how='left')

## merging to count the nbr of hhm
df_r1 = df_r1.merge(df_r1_SE.groupby(['hh_id'], as_index=False).size(), how="left", on=["hh_id"]).rename(columns={'size':'hh_size'}) 
df_r2 = df_r2.merge(df_r2_SE.groupby(['hh_id'], as_index=False).size(), how="left", on=["hh_id"]).rename(columns={'size':'hh_size'}) 
df_r3 = df_r3.merge(df_r3_SE.groupby(['hh_id'], as_index=False).size(), how="left", on=["hh_id"]).rename(columns={'size':'hh_size'}) 

df_r1["hh_size"] = df_r1["hh_size"].astype("int64", errors='ignore')
df_r2["hh_size"] = df_r2["hh_size"].astype("int64", errors='ignore')
df_r3["hh_size"] = df_r3["hh_size"].astype("int64", errors='ignore')


# merging to collect the same informations in r1 and r3
df_r3 = df_r3.merge(df_r1[['hh_id','district_name','upazila_name','union_name','union_code','code adm','affected_upazila_all','affected_komen', 'affected_roanu', 'affected_mora']], 
                    right_on='hh_id', left_on='hh_id_parent', how='left', suffixes=('_left', '_right')).rename(columns={'hh_id_left':'hh_id', 'union_code_right':'union_code'}).drop(columns=['hh_id_right', 'union_code_left'], axis=1)


df_r1["treat_group_all"] = 0
df_r2["treat_group_all"] = df_r2.apply(lambda x: 1 if (x["affected_upazila_all"]==1) & (x["interview_status"]=="Complete") else 0 if (x["affected_upazila_all"]==0) & (x["interview_status"]=="Complete") else np.nan, axis=1)
df_r3["treat_group_all"] = df_r3.apply(lambda x: 1 if (x["affected_upazila_all"]==1) & (x["interview_status"]=="Complete") else 0 if (x["affected_upazila_all"]==0) & (x["interview_status"]=="Complete") else np.nan, axis=1)

df_r1["treat_group_komen"] = 0
df_r2["treat_group_komen"] = df_r2.apply(lambda x: 1 if (x["affected_komen"]==1) & (x["interview_status"]=="Complete") else 0 if (x["affected_komen"]==0) & (x["interview_status"]=="Complete") else np.nan, axis=1)
df_r3["treat_group_komen"] = df_r3.apply(lambda x: 1 if (x["affected_komen"]==1) & (x["interview_status"]=="Complete") else 0 if (x["affected_komen"]==0) & (x["interview_status"]=="Complete") else np.nan, axis=1)

df_r1["treat_group_roanu"] = 0
df_r2["treat_group_roanu"] = df_r2.apply(lambda x: 1 if (x["affected_roanu"]==1) & (x["interview_status"]=="Complete") else 0 if (x["affected_roanu"]==0) & (x["interview_status"]=="Complete") else np.nan, axis=1)
df_r3["treat_group_roanu"] = df_r3.apply(lambda x: 1 if (x["affected_roanu"]==1) & (x["interview_status"]=="Complete") else 0 if (x["affected_roanu"]==0) & (x["interview_status"]=="Complete") else np.nan, axis=1)

df_r1["treat_group_mora"] = 0
df_r2["treat_group_mora"] = df_r2.apply(lambda x: 1 if (x["affected_mora"]==1) & (x["interview_status"]=="Complete") else 0 if (x["affected_mora"]==0) & (x["interview_status"]=="Complete") else np.nan, axis=1)
df_r3["treat_group_mora"] = df_r3.apply(lambda x: 1 if (x["affected_mora"]==1) & (x["interview_status"]=="Complete") else 0 if (x["affected_mora"]==0) & (x["interview_status"]=="Complete") else np.nan, axis=1)


df_r1["survey_year"] = 2011
df_r2["survey_year"] = 2015
df_r3["survey_year"] = 2018

In [41]:
# First merge to obtain the dataset for adults
## Merging to obtains the anthro and the informations on age, sex and id
df_r1_anthr1 = df_r1_anthr1.merge(df_r1_SE, on=['hh_id','hhm_id'], how='left')
df_r2_anthr1 = df_r2_anthr1.merge(df_r2_SE, on=['hh_id','hhm_id'], how='left')
df_r3_anthr1 = df_r3_anthr1.merge(df_r3_SE, on=['hh_id','hhm_id'], how='left')

## Merging to obtains the information on the household localisation (division, district, upazilla, union, moza, village)
df_r1_anthr1 = df_r1_anthr1.merge(df_r1, on=['hh_id'], how='left').drop(["index_x","index_y","member_id_y"], axis=1).rename(columns={"member_id_x":"member_id"})
df_r2_anthr1 = df_r2_anthr1.merge(df_r2, on=['hh_id'], how='left').drop(["member_id_y"], axis=1).rename(columns={"member_id_x":"member_id"})
df_r3_anthr1 = df_r3_anthr1.merge(df_r3, on=['hh_id'], how='left').drop(["member_id_y"], axis=1).rename(columns={"member_id_x":"member_id"})


# Second merge to obtain the dataset for children
## Merging to obtains the anthro and the informations on age, sex and id
df_r1_anthr2 = df_r1_anthr2.merge(df_r1_SE, on=['hh_id','hhm_id'], how='left')
df_r2_anthr2 = df_r2_anthr2.merge(df_r2_SE, on=['hh_id','hhm_id'], how='left')
df_r3_anthr2 = df_r3_anthr2.merge(df_r3_SE, on=['hh_id','hhm_id'], how='left')

## Merging to obtains the information on the household localisation (division, district, upazilla, union, moza, village)
df_r1_anthr2 = df_r1_anthr2.merge(df_r1, on=['hh_id'], how='left').drop(["index_x","index_y","member_id_y"], axis=1).rename(columns={"member_id_x":"member_id"})
df_r2_anthr2 = df_r2_anthr2.merge(df_r2, on=['hh_id'], how='left').drop(["member_id_y"], axis=1).rename(columns={"member_id_x":"member_id"})
df_r3_anthr2 = df_r3_anthr2.merge(df_r3, on=['hh_id'], how='left').drop(["member_id_y"], axis=1).rename(columns={"member_id_x":"member_id"})

In [42]:
# Computing age in days by subtracting the birth date from the interview date
df_r1_anthr2["age_days"] = (df_r1_anthr2.date_int - df_r1_anthr2.date_birth).dt.days

# Mapping household member sex to strings "male" and "female"
df_r1_anthr2["hhm_sex"] = df_r1_anthr2["hhm_sex"].map({"male": "male", "female": "female"})

df_r2_anthr1["hhm_sex"] = df_r2_anthr1["hhm_sex"].map({1: "male", 2: "female"})
df_r2_anthr2["hhm_sex"] = df_r2_anthr2["hhm_sex"].map({1: "male", 2: "female"})

df_r3_anthr1["hhm_sex"] = df_r3_anthr1["hhm_sex"].map({1: "male", 2: "female"})
df_r3_anthr2["hhm_sex"] = df_r3_anthr2["hhm_sex"].map({1: "male", 2: "female"})

In [43]:
df_r2_anthr2.hhm_status.value_counts()

hhm_status
New member (new born)                1674
Previous and current round member     914
Other reasons (permanent)             128
Household merged/combined              47
Name: count, dtype: int64

In [44]:
df_r3_anthr1.hhm_status.value_counts()

hhm_status
Previous and current round member                    18782
Other reasons (permanent)                             1242
New member through marriage                            609
Household merged/combined                              107
New member (new born)                                   18
New member upon return from divorce or seperation       14
Name: count, dtype: int64

### Unborn children status

In [45]:
df_r3_anthr2["if_not_measured_why"].value_counts()

if_not_measured_why
have measured    2277
absent             64
refused             9
sick                3
other               1
missing             0
Name: count, dtype: int64

In [46]:
lst_1 = df_r1.hh_id.values

lst_2 = df_r2.hh_id_parent.values

diff = list(set(lst_1) - set(lst_2))

diff

[]

In [47]:
## merging with the crosswalk
#df_r1_anthr1 = df_r1_anthr1.merge(crosswalk, on=['upazila_code'], how='left')
#df_r2_anthr1 = df_r2_anthr1.merge(crosswalk, on=['upazila_code'], how='left')
#df_r3_anthr1 = df_r3_anthr1.merge(crosswalk, on=['upazila_code'], how='left')

## merging with the crosswalk
#df_r1_anthr2 = df_r1_anthr2.merge(crosswalk, on=['upazila_code'], how='left')
#df_r2_anthr2 = df_r2_anthr2.merge(crosswalk, on=['upazila_code'], how='left')
#df_r3_anthr2 = df_r3_anthr2.merge(crosswalk, on=['upazila_code'], how='left')



In [48]:
#### Status of households surveyed (migrated)
print(df_r2[df_r2.sample_type!=2].drop_duplicates(subset=['hh_id_parent'], keep='first').groupby(['affected_upazila_all', 'interview_status'], observed=False)['hh_id_parent'].agg(['count']))

                                       count
affected_upazila_all interview_status       
0                    Complete           3877
                     Migrated            146
                     Not at home          16
                     Refused               4
1                    Complete           1350
                     Migrated             54
                     Not at home          10
                     Refused               6


In [49]:
#### Status of households surveyed (migrated)
# df_r2[df_r2.interview_status=="Completed"].groupby(['affected_upazila','hh_split']).agg(nbr_splited=("hh_split","count"))
#print(df_r2.groupby(['affected_upazila', 'interview_status'], observed=False)['hh_split'].agg(['count']))
# pd.crosstab(df_r2.affected_upazila, df_r2.hh_split, margins=True, normalize="index", rownames=['Affected upazila'], colnames=['Hh split'])

In [50]:
#### Status of households surveyed (migrated)
print(df_r3.groupby(['affected_upazila_all', 'interview_status'], observed=False)['hh_id'].agg(['count']))

                                       count
affected_upazila_all interview_status       
0                    Complete           4129
                     Refused               9
                     Not at home          14
                     Migrated            271
                     Partial               1
1                    Complete           1475
                     Refused              13
                     Not at home          10
                     Migrated             89
                     Partial               0


#### Building the global dataframe (for the 3 survey round)

In [51]:
df_r3_anthr1.hhm_status.value_counts()

hhm_status
Previous and current round member                    18782
Other reasons (permanent)                             1242
New member through marriage                            609
Household merged/combined                              107
New member (new born)                                   18
New member upon return from divorce or seperation       14
Name: count, dtype: int64

In [52]:
# here i keep only the hh with completed the survey and don't split during the study. Set remove the duplicates in the list
lst_hhm_id_r3 = np.unique(np.concatenate((df_r3_anthr1[(df_r3_anthr1.interview_status=="Complete") & (df_r3_anthr1.hh_split=="0") & (df_r3_anthr1.hhm_age < 20) & (df_r3_anthr1.hhm_status=="Previous and current round member")]["member_id"].values, 
                                        df_r3_anthr2[(df_r3_anthr2.interview_status=="Complete") & (df_r3_anthr2.hh_split=="0") & (df_r3_anthr2.hhm_age >= 3) & (df_r3_anthr2.hhm_status=="Previous and current round member")]["member_id"].values)))


# lst of underfive during the first or secind round
## fist (underfive first round); second (under 12 first round) and underfive in round 2
# 9238 children not splitted and before the natural disaster
#lst_underfive_int =  np.unique(np.concatenate((df_r1_anthr2.member_id.values, 
#                                            df_r1_anthr1[df_r1_anthr1.hhm_age <= 12].member_id.values, 
#                                            df_r2_anthr2[(df_r2_anthr2.interview_status=="Complete") &  (df_r2_anthr2.hh_split=="0")].member_id.values)))
lst_intvar = ['union_code','union_name','hh_id', 'hhm_id','weight_kg','height_cm','bmi' ,'if_not_measured_why', 'member_id','hhm_sex','hhm_status', 'hhm_age', 'relation_hhh', 'marital_status_hhm','hh_size',
    'literacy_hhm', 'education_high', 'curr_att_school', 'hh_head_religion','hh_ethnic_group','nbr_female', 'nbr_underfive',
    'nbr_yngchldrn_5_10', 'nbr_teenager_10_20', 'nbr_adults_20_65','nbr_elderly_65_over', 'nbr_female_underfive','nbr_female_yngchldrn_5_10', 
    'nbr_female_teenager_10_20','nbr_female_adults_20_65', 'nbr_female_elderly_65_over','sample_type', 'survey_round', 'code adm','affected_upazila_all','affected_komen', 'affected_roanu', 'affected_mora',]

#df_r1_global = pd.concat([df_r1_anthr1[df_r1_anthr1.member_id.isin(lst_hhm_id_r3)][lst_intvar].reset_index(drop=True), 
#                    df_r1_anthr2[df_r1_anthr2.member_id.isin(lst_hhm_id_r3)][lst_intvar].reset_index(drop=True)], ignore_index = True, verify_integrity=True)

#df_r2_global = pd.concat([df_r2_anthr1[df_r2_anthr1.member_id.isin(lst_hhm_id_r3)][lst_intvar].reset_index(drop=True),
#                    df_r2_anthr2[df_r2_anthr2.member_id.isin(lst_hhm_id_r3)][lst_intvar].reset_index(drop=True)], ignore_index = True, verify_integrity=True)

#df_r3_global = pd.concat([df_r3_anthr1[df_r3_anthr1.member_id.isin(lst_hhm_id_r3)][lst_intvar].reset_index(drop=True),
#                    df_r3_anthr2[df_r3_anthr2.member_id.isin(lst_hhm_id_r3)][lst_intvar].reset_index(drop=True)], ignore_index = True, verify_integrity=True)

df_global_corrected = pd.concat([pd.concat([df_r1_anthr1[df_r1_anthr1.member_id.isin(lst_hhm_id_r3)][lst_intvar].reset_index(drop=True), 
                    df_r1_anthr2[df_r1_anthr2.member_id.isin(lst_hhm_id_r3)][lst_intvar].reset_index(drop=True)], ignore_index = True, verify_integrity=True), 

                    pd.concat([df_r2_anthr1[df_r2_anthr1.member_id.isin(lst_hhm_id_r3)][lst_intvar].reset_index(drop=True),
                    df_r2_anthr2[df_r2_anthr2.member_id.isin(lst_hhm_id_r3)][lst_intvar].reset_index(drop=True)], ignore_index = True, verify_integrity=True), 
                    
                    pd.concat([df_r3_anthr1[df_r3_anthr1.member_id.isin(lst_hhm_id_r3)][lst_intvar].reset_index(drop=True),
                    df_r3_anthr2[df_r3_anthr2.member_id.isin(lst_hhm_id_r3)][lst_intvar].reset_index(drop=True)], ignore_index = True, verify_integrity=True)], ignore_index = True, verify_integrity=True)


df_global = pd.concat([pd.concat([df_r1_anthr1[lst_intvar].reset_index(drop=True), 
                    df_r1_anthr2[lst_intvar].reset_index(drop=True)], ignore_index = True, verify_integrity=True), 

                    pd.concat([df_r2_anthr1[lst_intvar].reset_index(drop=True),
                    df_r2_anthr2[lst_intvar].reset_index(drop=True)], ignore_index = True, verify_integrity=True), 
                    
                    pd.concat([df_r3_anthr1[lst_intvar].reset_index(drop=True),
                    df_r3_anthr2[lst_intvar].reset_index(drop=True)], ignore_index = True, verify_integrity=True)], ignore_index = True, verify_integrity=True)

# treatment binaries
df_global["treat_group_all"] = df_global.apply(lambda x: 1 if (x["survey_round"]=="3") & (x["affected_upazila_all"]==1) else 0, axis=1)
df_global["treat_komen"] = df_global.apply(lambda x: 1 if (x["survey_round"]=="3") & (x["affected_komen"]==1) else 0, axis=1)
df_global["treat_roanu"] = df_global.apply(lambda x: 1 if (x["survey_round"]=="3") & (x["affected_roanu"]==1) else 0, axis=1)
df_global["treat_mora"] = df_global.apply(lambda x: 1 if (x["survey_round"]=="3") & (x["affected_mora"]==1) else 0, axis=1)


df_global_corrected["treat_group_all"] = df_global_corrected.apply(lambda x: 1 if (x["survey_round"]=="3") & (x["affected_upazila_all"]==1) else 0, axis=1)
df_global_corrected["treat_komen"] = df_global_corrected.apply(lambda x: 1 if (x["survey_round"]=="3") & (x["affected_komen"]==1) else 0, axis=1)
df_global_corrected["treat_roanu"] = df_global_corrected.apply(lambda x: 1 if (x["survey_round"]=="3") & (x["affected_roanu"]==1) else 0, axis=1)
df_global_corrected["treat_mora"] = df_global_corrected.apply(lambda x: 1 if (x["survey_round"]=="3") & (x["affected_mora"]==1) else 0, axis=1)


# survey year
df_global["survey_year"] = df_global.apply(lambda x: 2018 if x["survey_round"]=="3" else 2015 if x["survey_round"]=="2" else 2011, axis=1)
df_global_corrected["survey_year"] = df_global_corrected.apply(lambda x: 2018 if x["survey_round"]=="3" else 2015 if x["survey_round"]=="2" else 2011, axis=1)


Data on FTF additional and National Representative data was collected during the third round of the BIHS. Data on FTF original was not collected, so i remove then from the previous rounds of the BIHS.

In [53]:
df_r3_anthr1.hhm_status.value_counts()

hhm_status
Previous and current round member                    18782
Other reasons (permanent)                             1242
New member through marriage                            609
Household merged/combined                              107
New member (new born)                                   18
New member upon return from divorce or seperation       14
Name: count, dtype: int64

In [54]:
test=df_global.groupby(["member_id"], observed=False)["survey_round"].agg(["sum"])
#test[].shape()
test["sum"] = test["sum"].astype("int64")
testview= test[ test["sum"] >= 4]
#print(df_r3.groupby(['affected_upazila', 'interview_status'], observed=False)['hh_id'].agg(['count']))

In [55]:
print(df_global.groupby(["survey_round"], observed=False).size())

df_r1_anthr1[lst_intvar]

survey_round
1    27285
2    27354
3    23126
dtype: int64


,union_code,union_name,hh_id,hhm_id,weight_kg,height_cm,bmi,if_not_measured_why,member_id,hhm_sex,...,nbr_female_teenager_10_20,nbr_female_adults_20_65,nbr_female_elderly_65_over,sample_type,survey_round,code adm,affected_upazila_all,affected_komen,affected_roanu,affected_mora
0,010817,Barai Para,0001,01,56.700001,164.800003,20.877026,have measured,000101,male,...,0,1,0,FTF additional,1,246,0,0,0,0
1,010817,Barai Para,0001,02,45.900002,143.300003,22.352201,have measured,000102,female,...,0,1,0,FTF additional,1,246,0,0,0,0
2,010817,Barai Para,0001,03,58.599998,169.300003,20.444838,have measured,000103,male,...,0,1,0,FTF additional,1,246,0,0,0,0
3,010817,Barai Para,0001,04,54.000000,170.300003,18.619347,have measured,000104,male,...,0,1,0,FTF additional,1,246,0,0,0,0
4,010817,Barai Para,0002,01,63.200001,168.399994,22.286042,have measured,000201,male,...,0,1,0,FTF additional,1,246,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24369,857309,Annadanagar,6501,03,15.500000,100.000000,15.500000,have measured,650103,female,...,0,1,0,National Representative,1,420,0,0,0,0
24370,857309,Annadanagar,6502,01,48.599998,159.000000,19.223923,have measured,650201,male,...,0,1,0,National Representative,1,420,0,0,0,0
24371,857309,Annadanagar,6502,02,50.000000,148.600006,22.642916,have measured,650202,female,...,0,1,0,National Representative,1,420,0,0,0,0
24372,857309,Annadanagar,6503,01,43.799999,160.500000,17.002940,have measured,650301,male,...,0,1,0,National Representative,1,420,0,0,0,0


### Importing to obtain exposure information

In [56]:


#Importing other shapefiles

# crosswalk

crosswalk = pd.read_excel(os.getcwd() + '\\input\\crosswalk\\crosswalk_union.xlsx', usecols="A:G", dtype={'union_name':'str','union_code_r1':'str','union_code_r2':'str','union_code_r3':'str',
                                                                                        'code_union_adm':'str','union_name_adm':'str','surveyed':'str'}) 


# Importing the Bangladesh raw map: Reading a shapefile containing administrative boundaries of Bangladesh
bgd_adm = gpd.read_file(os.getcwd() + '\\input\\shapefile_data\\shapefile_zip\\gadm41_BGD_shp\\gadm41_BGD_4.shp')


## Raining data (4 days before and after landfall)
raining_dta_komen = pd.read_stata(os.getcwd() + '\\input\\shapefile_data\\other data\\bgd\\bgd\\BGD4_mswep_2010_2019.dta', 
                        columns=["GID_4","M2015205","M2015206","M2015207","M2015208","M2015209","M2015210","M2015211","M2015212","M2015213"])

raining_dta_roanu = pd.read_stata(os.getcwd() + '\\input\\shapefile_data\\other data\\bgd\\bgd\\BGD4_mswep_2010_2019.dta', 
                        columns=["GID_4","M2016137","M2016138","M2016139","M2016140","M2016141","M2016142","M2016143","M2016144","M2016145"])

raining_dta_mora = pd.read_stata(os.getcwd() + '\\input\\shapefile_data\\other data\\bgd\\bgd\\BGD4_mswep_2010_2019.dta', 
                        columns=["GID_4","M2017145","M2017146","M2017147","M2017148","M2017149","M2017150","M2017151","M2017152","M2017153"])

## NTL data (4 days before and after landfall)
ntl_dta_komen = pd.read_stata(os.getcwd() + '\\input\\shapefile_data\\other data\\bgd\\bgd\\BGD4_ntl_bm_2013_2019.dta', 
                        columns=["gid_4","n2015_07_25","n2015_07_26","n2015_07_27","n2015_07_28","n2015_07_29","n2015_07_30","n2015_07_31","n2015_08_01","n2015_08_02"])

ntl_dta_roanu = pd.read_stata(os.getcwd() + '\\input\\shapefile_data\\other data\\bgd\\bgd\\BGD4_ntl_bm_2013_2019.dta', 
                        columns=["gid_4","n2016_05_17","n2016_05_18","n2016_05_19","n2016_05_20","n2016_05_21","n2016_05_22","n2016_05_23","n2016_05_24","n2016_05_25"])

ntl_dta_mora = pd.read_stata(os.getcwd() + '\\input\\shapefile_data\\other data\\bgd\\bgd\\BGD4_ntl_bm_2013_2019.dta', 
                        columns=["gid_4","n2017_05_26","n2017_05_27","n2017_05_28","n2017_05_29","n2017_05_30","n2017_05_31","n2017_06_01","n2017_06_02","n2017_06_03"])

## Pop density data
popdens_dta = pd.read_csv(os.getcwd() + '\\input\\shapefile_data\\other data\\bgd\\bgd\\bgd4_pop_landscan_2000_2020.csv',sep=";", usecols=["GID_4","landscan_global_2017"])

## Storm data
exposure_time_komen_W_N = DBF(os.getcwd() + '\\input\\shapefile_data\\other data\\bgd\\bgd\\stats_stormR\\exposure_msw_komen_W_N.dbf', load=True)
exposure_time_roanu_W_N = DBF(os.getcwd() + '\\input\\shapefile_data\\other data\\bgd\\bgd\\stats_stormR\\exposure_msw_roanu_W_N.dbf', load=True)
exposure_time_mora_W_N = DBF(os.getcwd() + '\\input\\shapefile_data\\other data\\bgd\\bgd\\stats_stormR\\exposure_msw_mora_W_N.dbf', load=True)

speed_komen_W_N = DBF(os.getcwd() + '\\input\\shapefile_data\\other data\\bgd\\bgd\\stats_stormR\\msw_komen_W_N.dbf', load=True)
speed_roanu_W_N = DBF(os.getcwd() + '\\input\\shapefile_data\\other data\\bgd\\bgd\\stats_stormR\\msw_roanu_W_N.dbf', load=True)
speed_mora_W_N = DBF(os.getcwd() + '\\input\\shapefile_data\\other data\\bgd\\bgd\\stats_stormR\\msw_mora_W_N.dbf', load=True)

# Convert to Dataframe
exposure_time_komen_W_N = pd.DataFrame(iter(exposure_time_komen_W_N))
exposure_time_roanu_W_N = pd.DataFrame(iter(exposure_time_roanu_W_N))
exposure_time_mora_W_N = pd.DataFrame(iter(exposure_time_mora_W_N))

speed_komen_W_N = pd.DataFrame(iter(speed_komen_W_N))
speed_roanu_W_N = pd.DataFrame(iter(speed_roanu_W_N))
speed_mora_W_N = pd.DataFrame(iter(speed_mora_W_N))

#Dropping and renaming columns
exposure_time_komen_W_N = exposure_time_komen_W_N[["GID_4","COUNT","AREA","MIN","MAX","MEAN"]]
exposure_time_roanu_W_N = exposure_time_roanu_W_N[["GID_4","COUNT","AREA","MIN","MAX","MEAN"]]
exposure_time_mora_W_N = exposure_time_mora_W_N[["GID_4","COUNT","AREA","MIN","MAX","MEAN"]]

speed_komen_W_N = speed_komen_W_N[["GID_4","COUNT","AREA","MIN","MAX","MEAN"]]
speed_roanu_W_N = speed_roanu_W_N[["GID_4","COUNT","AREA","MIN","MAX","MEAN"]]
speed_mora_W_N = speed_mora_W_N[["GID_4","COUNT","AREA","MIN","MAX","MEAN"]]

exposure_time_komen_W_N.rename(columns={"COUNT":"count_time_komen","AREA":"area_time_komen","MIN":"min_time_komen","MAX":"max_time_komen","MEAN":"mean_time_komen"}, inplace=True)
exposure_time_roanu_W_N.rename(columns={"COUNT":"count_time_roanu","AREA":"area_time_roanu","MIN":"min_time_roanu","MAX":"max_time_roanu","MEAN":"mean_time_roanu"}, inplace=True)
exposure_time_mora_W_N.rename(columns={"COUNT":"count_time_mora","AREA":"area_time_mora","MIN":"min_time_mora","MAX":"max_time_mora","MEAN":"mean_time_mora"}, inplace=True)

speed_komen_W_N.rename(columns={"COUNT":"count_speed_komen","AREA":"area_speed_komen","MIN":"min_speed_komen","MAX":"max_speed_komen","MEAN":"mean_speed_komen"}, inplace=True)
speed_roanu_W_N.rename(columns={"COUNT":"count_speed_roanu","AREA":"area_speed_roanu","MIN":"min_speed_roanu","MAX":"max_speed_roanu","MEAN":"mean_speed_roanu"}, inplace=True)
speed_mora_W_N.rename(columns={"COUNT":"count_speed_mora","AREA":"area_speed_mora","MIN":"min_speed_mora","MAX":"max_speed_mora","MEAN":"mean_speed_mora"}, inplace=True)


## Merging with the crosswalk to get surveyed unions

bgd_adm =(
    crosswalk
    
    # Komen
    .merge(bgd_adm, how='right', left_on="code_union_adm", right_on="GID_4")
    .merge(raining_dta_komen, how='left', left_on="GID_4", right_on="GID_4")
    .merge(ntl_dta_komen, how='left', left_on="GID_4", right_on="gid_4")
    .merge(popdens_dta, how='left', left_on="GID_4", right_on="GID_4")
    .merge(exposure_time_komen_W_N, how='left', left_on="GID_4", right_on="GID_4")
    .merge(speed_komen_W_N, how='left', left_on="GID_4", right_on="GID_4")

    # Roanu
    .merge(raining_dta_roanu, how='left', left_on="GID_4", right_on="GID_4")
    .merge(ntl_dta_roanu, how='left', left_on="GID_4", right_on="gid_4")
    .merge(exposure_time_roanu_W_N, how='left', left_on="GID_4", right_on="GID_4")
    .merge(speed_roanu_W_N, how='left', left_on="GID_4", right_on="GID_4")

    # Mora
    .merge(raining_dta_mora, how='left', left_on="GID_4", right_on="GID_4")
    .merge(ntl_dta_mora, how='left', left_on="GID_4", right_on="gid_4")
    .merge(exposure_time_mora_W_N, how='left', left_on="GID_4", right_on="GID_4")
    .merge(speed_mora_W_N, how='left', left_on="GID_4", right_on="GID_4")
    )


bgd_adm["surveyed"] = bgd_adm["surveyed"].fillna("0").replace({'1':'Surveyed union','0':'Unsurveyed union'}).astype(str)
bgd_adm["union_code_r1"] = bgd_adm["union_code_r1"].astype(str)
bgd_adm["union_name"] = bgd_adm["union_name"].astype(str)

bgd_adm["average_rain_komen"] = bgd_adm[["M2015208","M2015209","M2015210"]].aggregate("mean",axis="columns")
bgd_adm["average_rain_roanu"] = bgd_adm[["M2016140","M2016141","M2016142"]].aggregate("mean",axis="columns")
bgd_adm["average_rain_mora"] = bgd_adm[["M2017148","M2017149","M2017150"]].aggregate("mean",axis="columns")

bgd_adm["average_ntl_komen"] = bgd_adm[["n2015_07_28","n2015_07_29","n2015_07_30"]].aggregate("mean",axis="columns")
bgd_adm["average_ntl_roanu"] = bgd_adm[["n2016_05_20","n2016_05_21","n2016_05_22"]].aggregate("mean",axis="columns")
bgd_adm["average_ntl_mora"] = bgd_adm[["n2017_05_29","n2017_05_30","n2017_05_31"]].aggregate("mean",axis="columns")


bgd_adm = gpd.GeoDataFrame(bgd_adm, geometry='geometry')

# Building quantiles for highly exposed and unexposed regions
quantiles_high_komen = np.nanquantile(bgd_adm[bgd_adm['mean_speed_komen']>0].mean_speed_komen, [i * 0.25 + 0.25 for i in range(4)]) 
bgd_adm["high_exposed_komen"] = bgd_adm["mean_speed_komen"].apply(lambda row: "1" if row >= int(quantiles_high_komen[2]) else "0")
bgd_adm["exposed_at_least_zero_komen"] = bgd_adm["mean_speed_komen"].apply(lambda row: "1" if row > 0 else "0")

quantiles_high_roanu = np.nanquantile(bgd_adm[bgd_adm['mean_speed_roanu']>0].mean_speed_roanu, [i * 0.25 + 0.25 for i in range(4)]) 
bgd_adm["high_exposed_roanu"] = bgd_adm["mean_speed_roanu"].apply(lambda row: "1" if row >= int(quantiles_high_roanu[2]) else "0")
bgd_adm["exposed_at_least_zero_roanu"] = bgd_adm["mean_speed_roanu"].apply(lambda row: "1" if row > 0 else "0")

quantiles_high_mora = np.nanquantile(bgd_adm[bgd_adm['mean_speed_mora']>0].mean_speed_mora, [i * 0.25 + 0.25 for i in range(4)]) 
bgd_adm["high_exposed_mora"] = bgd_adm["mean_speed_mora"].apply(lambda row: "1" if row >= int(quantiles_high_mora[2]) else "0")
bgd_adm["exposed_at_least_zero_mora"] = bgd_adm["mean_speed_mora"].apply(lambda row: "1" if row > 0 else "0")



In [57]:
# removing already existing object for an efficient usage of the computer memory

del raining_dta_komen
del ntl_dta_komen
del raining_dta_roanu
del ntl_dta_roanu
del raining_dta_mora
del ntl_dta_mora
del popdens_dta
del exposure_time_komen_W_N
del speed_komen_W_N
del exposure_time_roanu_W_N
del speed_roanu_W_N
del exposure_time_mora_W_N
del speed_mora_W_N

#### Merging adm data with survey data 

In [58]:
# Round 1 data
df_r1 = df_r1.merge(
        bgd_adm[['union_name', 'union_code_r1', 'code_union_adm', 'union_name_adm',
        'high_exposed_komen', 'exposed_at_least_zero_komen','mean_speed_komen','mean_time_komen', 
        'high_exposed_roanu', 'exposed_at_least_zero_roanu','mean_speed_roanu','mean_time_roanu',
        'high_exposed_mora', 'exposed_at_least_zero_mora','mean_speed_mora','mean_time_mora']],
        how='left',
        left_on=['union_name', 'union_code'],
        right_on=['union_name', 'union_code_r1'])

df_r1_anthr1 = df_r1_anthr1.merge(
        bgd_adm[['union_name', 'union_code_r1', 'code_union_adm', 'union_name_adm',
        'high_exposed_komen', 'exposed_at_least_zero_komen','mean_speed_komen','mean_time_komen', 
        'high_exposed_roanu', 'exposed_at_least_zero_roanu','mean_speed_roanu','mean_time_roanu',
        'high_exposed_mora', 'exposed_at_least_zero_mora','mean_speed_mora','mean_time_mora']],
        how='left',
        left_on=['union_name', 'union_code'],
        right_on=['union_name', 'union_code_r1'])

df_r1_anthr2 = df_r1_anthr2.merge(
        bgd_adm[['union_name', 'union_code_r1', 'code_union_adm', 'union_name_adm',
        'high_exposed_komen', 'exposed_at_least_zero_komen','mean_speed_komen','mean_time_komen', 
        'high_exposed_roanu', 'exposed_at_least_zero_roanu','mean_speed_roanu','mean_time_roanu',
        'high_exposed_mora', 'exposed_at_least_zero_mora','mean_speed_mora','mean_time_mora']],
        how='left',
        left_on=['union_name', 'union_code'],
        right_on=['union_name', 'union_code_r1'])

# Round 2 data
df_r2 = df_r2.merge(
        bgd_adm[['union_name', 'union_code_r1', 'code_union_adm', 'union_name_adm',
        'high_exposed_komen', 'exposed_at_least_zero_komen','mean_speed_komen','mean_time_komen', 
        'high_exposed_roanu', 'exposed_at_least_zero_roanu','mean_speed_roanu','mean_time_roanu',
        'high_exposed_mora', 'exposed_at_least_zero_mora','mean_speed_mora','mean_time_mora']],
        how='left',
        left_on=['union_name', 'union_code'],
        right_on=['union_name', 'union_code_r1'])

df_r2_anthr1 = df_r2_anthr1.merge(
        bgd_adm[['union_name', 'union_code_r1', 'code_union_adm', 'union_name_adm',
        'high_exposed_komen', 'exposed_at_least_zero_komen','mean_speed_komen','mean_time_komen', 
        'high_exposed_roanu', 'exposed_at_least_zero_roanu','mean_speed_roanu','mean_time_roanu',
        'high_exposed_mora', 'exposed_at_least_zero_mora','mean_speed_mora','mean_time_mora']],
        how='left',
        left_on=['union_name', 'union_code'],
        right_on=['union_name', 'union_code_r1'])

df_r2_anthr2 = df_r2_anthr2.merge(
        bgd_adm[['union_name', 'union_code_r1', 'code_union_adm', 'union_name_adm',
        'high_exposed_komen', 'exposed_at_least_zero_komen','mean_speed_komen','mean_time_komen', 
        'high_exposed_roanu', 'exposed_at_least_zero_roanu','mean_speed_roanu','mean_time_roanu',
        'high_exposed_mora', 'exposed_at_least_zero_mora','mean_speed_mora','mean_time_mora']],
        how='left',
        left_on=['union_name', 'union_code'],
        right_on=['union_name', 'union_code_r1'])


# Round 3 data
df_r3 = df_r3.merge(
        bgd_adm[['union_name', 'union_code_r1', 'code_union_adm', 'union_name_adm',
        'high_exposed_komen', 'exposed_at_least_zero_komen','mean_speed_komen','mean_time_komen', 
        'high_exposed_roanu', 'exposed_at_least_zero_roanu','mean_speed_roanu','mean_time_roanu',
        'high_exposed_mora', 'exposed_at_least_zero_mora','mean_speed_mora','mean_time_mora']],
        how='left',
        left_on=['union_name', 'union_code'],
        right_on=['union_name', 'union_code_r1'])

df_r3_anthr1 = df_r3_anthr1.merge(
        bgd_adm[['union_name', 'union_code_r1', 'code_union_adm', 'union_name_adm',
        'high_exposed_komen', 'exposed_at_least_zero_komen','mean_speed_komen','mean_time_komen', 
        'high_exposed_roanu', 'exposed_at_least_zero_roanu','mean_speed_roanu','mean_time_roanu',
        'high_exposed_mora', 'exposed_at_least_zero_mora','mean_speed_mora','mean_time_mora']],
        how='left',
        left_on=['union_name', 'union_code'],
        right_on=['union_name', 'union_code_r1'])

df_r3_anthr2 = df_r3_anthr2.merge(
        bgd_adm[['union_name', 'union_code_r1', 'code_union_adm', 'union_name_adm',
        'high_exposed_komen', 'exposed_at_least_zero_komen','mean_speed_komen','mean_time_komen', 
        'high_exposed_roanu', 'exposed_at_least_zero_roanu','mean_speed_roanu','mean_time_roanu',
        'high_exposed_mora', 'exposed_at_least_zero_mora','mean_speed_mora','mean_time_mora']],
        how='left',
        left_on=['union_name', 'union_code'],
        right_on=['union_name', 'union_code_r1'])


# Correct a mistake in the name
#df_global_corrected['union_name'] = df_global_corrected['union_name'].replace({"Bhayna  (Sujanagar)":"Bhayna"})

df_global_corrected = df_global_corrected.merge(
        bgd_adm[['union_name', 'union_code_r1','code_union_adm', 'union_name_adm',
        'high_exposed_komen', 'exposed_at_least_zero_komen','mean_speed_komen','mean_time_komen', 
        'high_exposed_roanu', 'exposed_at_least_zero_roanu','mean_speed_roanu','mean_time_roanu',
        'high_exposed_mora', 'exposed_at_least_zero_mora','mean_speed_mora','mean_time_mora']],
        how='left',
        left_on=['union_name', 'union_code'],
        right_on=['union_name', 'union_code_r1'])


df_global = df_global.merge(
        bgd_adm[['union_name', 'union_code_r1','code_union_adm', 'union_name_adm',
        'high_exposed_komen', 'exposed_at_least_zero_komen','mean_speed_komen','mean_time_komen', 
        'high_exposed_roanu', 'exposed_at_least_zero_roanu','mean_speed_roanu','mean_time_roanu',
        'high_exposed_mora', 'exposed_at_least_zero_mora','mean_speed_mora','mean_time_mora']],
        how='left',
        left_on=['union_name', 'union_code'],
        right_on=['union_name', 'union_code_r1'])

# bgd adm information cleaning
bgd_adm["union_name_adm"] = bgd_adm["union_name_adm"].astype(str)
bgd_adm["union_name"] = bgd_adm["union_name"].astype(str)
bgd_adm["union_code_r1"] = bgd_adm["union_code_r1"].astype(str)
bgd_adm["union_code_r2"] = bgd_adm["union_code_r2"].astype(str)
bgd_adm["union_code_r3"] = bgd_adm["union_code_r3"].astype(str)


In [59]:
# correcting union_name before exporting

df_global_corrected['union_name'] = df_global_corrected['union_name'].astype(str)
#df_global_corrected['union_name_r1'] = df_global_corrected['union_name_r1'].astype(str)


df_global['union_name'] = df_global['union_name'].astype(str)
#df_global['union_name_r1'] = df_global['union_name_r1'].astype(str)


df_global['union_name'] = df_global['union_name'].astype(str)
#df_global['union_name_r1'] = df_global['union_name_r1'].astype(str)

### Export of the dataframes

In [60]:


# Saving the processed dataframes to Excel files

## Anthro for > 5 hhm
df_r1_anthr1.to_excel(os.getcwd() + "\\output\\data\\r1_overfive.xlsx", index=False)
df_r2_anthr1.to_excel(os.getcwd() + "\\output\\data\\r2_overfive.xlsx", index=False)
df_r3_anthr1.to_excel(os.getcwd() + "\\output\\data\\r3_overfive.xlsx", index=False)

## Anthro for <= 5 hhm
df_r1_anthr2.to_excel(os.getcwd() + "\\output\\data\\r1_underfive.xlsx", index=False)
df_r2_anthr2.to_excel(os.getcwd() + "\\output\\data\\r2_underfive.xlsx", index=False)
df_r3_anthr2.to_excel(os.getcwd() + "\\output\\data\\r3_underfive.xlsx", index=False)

## Hh level data  
df_r1.to_excel(os.getcwd() + "\\output\\data\\r1_hh.xlsx", index=False)
df_r2.to_excel(os.getcwd() + "\\output\\data\\r2_hh.xlsx", index=False)
df_r3.to_excel(os.getcwd() + "\\output\\data\\r3_hh.xlsx", index=False)

## Concatenated data

df_global_corrected.to_excel(os.getcwd() + "\\output\\data\\df_global_corrected.xlsx", index=False)
df_global.to_excel(os.getcwd() + "\\output\\data\\df_global.xlsx", index=False)
bgd_adm.to_csv(os.getcwd() + "\\output\\data\\bgd_data.csv", sep=";", index=False)
#bgd_adm.to_file(os.getcwd() + "\\output\\data\\bgd_data.shp")
bgd_adm.to_file(os.getcwd() + "\\output\\data\\bgd_data.geojson", driver="GeoJSON") 

In [61]:
import geopandas as gpd
import os 
import pandas as pd
# Importing the Bangladesh raw map: Reading a shapefile containing administrative boundaries of Bangladesh
bgd_adm = gpd.read_file(os.getcwd() + '\\input\\shapefile_data\\shapefile_zip\\gadm41_BGD_shp\\gadm41_BGD_4.shp')
raining_dta_komen = pd.read_stata(os.getcwd() + '\\input\\shapefile_data\\other data\\bgd\\bgd\\BGD4_mswep_2010_2019.dta', 
                        columns=["GID_4","M2015205","M2015206","M2015207","M2015208","M2015209","M2015210","M2015211","M2015212","M2015213"])


In [62]:
# Release memory using del
del crosswalk
del df_r1
del df_r1_f
del df_r1_ftf
del df_r1_SE
del df_r2
del df_r2_f
del df_r2_SE
del df_r3
del df_r3_f
del df_r3_SE
del df_global
del df_global_corrected

del df_r1_anthr1
del df_r1_anthr2
del df_r2_anthr1
del df_r2_anthr2
del df_r3_anthr1
del df_r3_anthr2

del nbr_max
del label_to_int
del lst_1
del lst_2
del diff
del lst_complete
del lst_migrate
del lst_refused
del lst_nthome
del lst_hhm_id_r3
del lst_intvar
del lst_split
del test
del testview
del lst_r3
del df_r1_SE_gr
del df_r2_SE_gr
del df_r3_SE_gr
# Only non nan values
#df_r1_anthr2=df_r1_anthr2[(df_r1_anthr2.weight_kg.notnull()) & (df_r1_anthr2.height_cm.notnull()) & (df_r1_anthr2.age_days.notnull())]
# df_r2_anthr2=df_r2_anthr2[(df_r2_anthr2.weight_kg.notnull()) & (df_r2_anthr2.height_cm.notnull()) & (df_r2_anthr2.age_days.notnull())]